# **Conjunto de dados 2**

Neste conjunto utilizaremos as perguntas de concurso como pergunta e resposta, e pediremos as LLMs para encontrar o contexto no código civil.

Para isso é preciso:

1. Processar as perguntas do concurso
    
    a) Perguntas identicas ao do concurso
    
    b) Quebrar em perguntas menores de SIM ou NÃO reduzindo o contexto.

1. Fazer uma espécie de RAG para encontrar os contextos

1. Avaliar os contextos encotrados
    
    3.1. - Podemos usar os comentários das perguntas para validar se os artigos citados nos comentários também são referenciados no contexto.

# ENV Setup

In [163]:
!pip install langchain-groq
!pip install llama-index-llms-groq
!pip install llama-index
!pip install llama-index-embeddings-huggingface

# # Install the mains crewAI package
# !pip install crewai

# # Install the main crewAI package and the tools package
# # that includes a series of helpful tools for your agents
# !pip install 'crewai[tools]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 5.0 MB/s eta 0:00:00


# Imports e Defines

In [151]:
import os
import json
import pandas as pd
#import random
#import numpy as np
from getpass import getpass
from tqdm import tqdm

from langchain_groq import ChatGroq
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

In [94]:
GROQ_API_KEY = getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [152]:
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

# Load de Modelos LLM

In [95]:
llama3_70b_temp0 = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",
    api_key=GROQ_API_KEY)

# Test
llama3_70b_temp0.invoke("What is the capital of France?")

AIMessage(content='The capital of France is Paris.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 17, 'total_tokens': 25, 'completion_time': 0.018941071, 'prompt_time': 0.005670547, 'queue_time': None, 'total_time': 0.024611618000000002}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_753a4aecf6', 'finish_reason': 'stop', 'logprobs': None}, id='run-fb552120-e141-4896-bfbb-a12986288e13-0')

# Processamento dos Dados

## Item (a) usar perguntas similares a concurso

Já possuímos 16 perguntas processadas. Vamos usá-las como experimento para a construção dos nosso algoritmos.

In [96]:
with open("/content/QAsites_FAMILIA_SUCESSOES.json", "r") as f:
    examples = json.load(f)

In [97]:
examples_df = pd.DataFrame(examples)
concurso_qa = examples_df[examples_df['name'].str.contains('granconcursos')].reset_index()

concurso_qa

,index,name,link,tema,pergunta,resposta
0,46,granconcursos_direitocivil_direitofamilia_dire...,https://questoes.grancursosonline.com.br/aluno...,Questões de Concursos - 1 a 15 - pagina 1,1 - Q3121863\nDireito Civil Sucessão Testamen...,I e IV
1,47,granconcursos_direitocivil_direitofamilia_dire...,https://questoes.grancursosonline.com.br/aluno...,Questões de Concursos - 1 a 15 - pagina 1,2 - Q3116412\nDireito Civil Direito das Suces...,C
2,48,granconcursos_direitocivil_direitofamilia_dire...,https://questoes.grancursosonline.com.br/aluno...,Questões de Concursos - 1 a 15 - pagina 1,3 - Q3251846\nDireito Civil Do Rompimento do ...,E
3,49,granconcursos_direitocivil_direitofamilia_dire...,https://questoes.grancursosonline.com.br/aluno...,Questões de Concursos - 1 a 15 - pagina 1,4 - Q3192677\nDireito Civil Fato Jurídico Nat...,C
4,50,granconcursos_direitocivil_direitofamilia_dire...,https://questoes.grancursosonline.com.br/aluno...,Questões de Concursos - 1 a 15 - pagina 1,5- Q3158359\nDireito Civil Sucessão Testament...,C
5,51,granconcursos_direitocivil_direitofamilia_dire...,https://questoes.grancursosonline.com.br/aluno...,Questões de Concursos - 1 a 15 - pagina 1,6 - Q3116405\nDireito Civil Direito das Suces...,E
6,52,granconcursos_direitocivil_direitofamilia_dire...,https://questoes.grancursosonline.com.br/aluno...,Questões de Concursos - 1 a 15 - pagina 1,7 - Q3231293\nDireito Civil Ordem Da Vocação ...,ERRADO
7,53,granconcursos_direitocivil_direitofamilia_dire...,https://questoes.grancursosonline.com.br/aluno...,Questões de Concursos - 1 a 15 - pagina 1,8 - Q3116416\nDireito Civil Direito de Famíli...,E
8,54,granconcursos_direitocivil_direitofamilia_dire...,https://questoes.grancursosonline.com.br/aluno...,Questões de Concursos - 1 a 15 - pagina 1,9 - Q3231292\nDireito Civil Direito das Suces...,ERRADO
9,55,granconcursos_direitocivil_direitofamilia_dire...,https://questoes.grancursosonline.com.br/aluno...,Questões de Concursos - 1 a 15 - pagina 1,10 - Q3069534\nDireito Civil Sucessão Testame...,A


In [98]:
sample = concurso_qa.loc[0, 'pergunta']
sample

'1 - Q3121863\nDireito Civil  Sucessão Testamentária (Art. 1.857 ao 1.990)\nAno: 2024\nBanca: Instituto Consulplan\nProva: Instituto Consulplan - DPE PR - Analista - Área Direito - 2024\nA sucessão testamentária é um componente importante do planejamento patrimonial e permite que as pessoas tenham controle sobre o destino de seus bens após a morte. É fundamental que um testamento seja elaborado de forma cuidadosa e de acordo com as leis locais para garantir que a vontade do testador seja respeitada e que a sucessão ocorra de maneira adequada. Em relação à sucessão testamentária, analise as afirmativas a seguir.\nI. Ao cego só se permite o testamento público, que lhe será lido, em voz alta, duas vezes, uma pelo tabelião ou por seu substituto legal, e a outra por uma das testemunhas, designada pelo testador, fazendo-se de tudo circunstanciada menção no testamento.\nII. O testamento cerrado deve ser escrito mecanicamente, desde que seu subscritor numere e autentique, com a sua assinatura,

In [99]:
new_row = {'id': None, 'area': None, 'ano':None, 'banca': None, 'prova': None, 'pergunta_texto': None}
new_rows = []
for sample in concurso_qa['pergunta']:
    splitted = sample.split('\n')
    new_row['id'] = splitted[0]
    new_row['area'] = splitted[1]
    new_row['ano'] = splitted[2]
    new_row['banca'] = splitted[3]
    new_row['prova'] = splitted[4]
    new_row['pergunta_texto'] = '\n'.join(splitted[5:])
    new_rows.append(new_row.copy())

append_df = pd.DataFrame(new_rows)
append_df

,id,area,ano,banca,prova,pergunta_texto
0,1 - Q3121863,Direito Civil Sucessão Testamentária (Art. 1....,Ano: 2024,Banca: Instituto Consulplan,Prova: Instituto Consulplan - DPE PR - Analist...,A sucessão testamentária é um componente impor...
1,2 - Q3116412,Direito Civil Direito das Sucessões (Art. 1.7...,Ano: 2024,Banca: Fundação Getúlio Vargas - FGV,Prova: FGV - Câmara de São Paulo - Procurador ...,"Diego, em razão do falecimento do seu pai Eucl..."
2,3 - Q3251846,Direito Civil Do Rompimento do Testamento,Ano: 2024,Banca: Fundação para o Vestibular da Universid...,Prova: VUNESP - MPE RO - Promotor de Justiça S...,"Matheus, filho de um milionário, aos dezesseis..."
3,4 - Q3192677,Direito Civil Fato Jurídico Natural Extraordi...,Ano: 2024,Banca: Fundação para o Vestibular da Universid...,Prova: VUNESP - MPE RJ - Promotor de Justiça S...,Rogério é casado com Cláudio e ambos são pais ...
4,5- Q3158359,Direito Civil Sucessão Testamentária (Art. 1....,Ano: 2024,Banca: Fundação Getúlio Vargas - FGV,Prova: FGV - MPE GO - Promotor de Justiça Subs...,"Mariana, solteira, não convivente em união est..."
5,6 - Q3116405,Direito Civil Direito das Sucessões (Art. 1.7...,Ano: 2024,Banca: Fundação Getúlio Vargas - FGV,Prova: FGV - Câmara de São Paulo - Procurador ...,Lofredo faleceu deixando quatro filhas vivas: ...
6,7 - Q3231293,Direito Civil Ordem Da Vocação Hereditária,Ano: 2024,Banca: Instituto Consulplan,Prova: Instituto Consulplan - MPE SC - Promoto...,"João e Maria, amigos de longa data, decidem em..."
7,8 - Q3116416,Direito Civil Direito de Família (Art. 1.511 ...,Ano: 2024,Banca: Fundação Getúlio Vargas - FGV,Prova: FGV - Câmara de São Paulo - Procurador ...,"Em março de 2003, Homero e Agatha se casaram p..."
8,9 - Q3231292,Direito Civil Direito das Sucessões (Art. 1.7...,Ano: 2024,Banca: Instituto Consulplan,Prova: Instituto Consulplan - MPE SC - Promoto...,"João e Maria, amigos de longa data, decidem em..."
9,10 - Q3069534,Direito Civil Sucessão Testamentária (Art. 1....,Ano: 2023,Banca: Fundação Getúlio Vargas - FGV,Prova: FGV - TJ PR - Juiz Substituto - 2023,"Marcelo tem deficiência visual em alto grau, q..."


In [100]:
concursos_df = pd.concat([concurso_qa, append_df], axis=1)
concursos_df

,index,name,link,tema,pergunta,resposta,id,area,ano,banca,prova,pergunta_texto
0,46,granconcursos_direitocivil_direitofamilia_dire...,https://questoes.grancursosonline.com.br/aluno...,Questões de Concursos - 1 a 15 - pagina 1,1 - Q3121863\nDireito Civil Sucessão Testamen...,I e IV,1 - Q3121863,Direito Civil Sucessão Testamentária (Art. 1....,Ano: 2024,Banca: Instituto Consulplan,Prova: Instituto Consulplan - DPE PR - Analist...,A sucessão testamentária é um componente impor...
1,47,granconcursos_direitocivil_direitofamilia_dire...,https://questoes.grancursosonline.com.br/aluno...,Questões de Concursos - 1 a 15 - pagina 1,2 - Q3116412\nDireito Civil Direito das Suces...,C,2 - Q3116412,Direito Civil Direito das Sucessões (Art. 1.7...,Ano: 2024,Banca: Fundação Getúlio Vargas - FGV,Prova: FGV - Câmara de São Paulo - Procurador ...,"Diego, em razão do falecimento do seu pai Eucl..."
2,48,granconcursos_direitocivil_direitofamilia_dire...,https://questoes.grancursosonline.com.br/aluno...,Questões de Concursos - 1 a 15 - pagina 1,3 - Q3251846\nDireito Civil Do Rompimento do ...,E,3 - Q3251846,Direito Civil Do Rompimento do Testamento,Ano: 2024,Banca: Fundação para o Vestibular da Universid...,Prova: VUNESP - MPE RO - Promotor de Justiça S...,"Matheus, filho de um milionário, aos dezesseis..."
3,49,granconcursos_direitocivil_direitofamilia_dire...,https://questoes.grancursosonline.com.br/aluno...,Questões de Concursos - 1 a 15 - pagina 1,4 - Q3192677\nDireito Civil Fato Jurídico Nat...,C,4 - Q3192677,Direito Civil Fato Jurídico Natural Extraordi...,Ano: 2024,Banca: Fundação para o Vestibular da Universid...,Prova: VUNESP - MPE RJ - Promotor de Justiça S...,Rogério é casado com Cláudio e ambos são pais ...
4,50,granconcursos_direitocivil_direitofamilia_dire...,https://questoes.grancursosonline.com.br/aluno...,Questões de Concursos - 1 a 15 - pagina 1,5- Q3158359\nDireito Civil Sucessão Testament...,C,5- Q3158359,Direito Civil Sucessão Testamentária (Art. 1....,Ano: 2024,Banca: Fundação Getúlio Vargas - FGV,Prova: FGV - MPE GO - Promotor de Justiça Subs...,"Mariana, solteira, não convivente em união est..."
5,51,granconcursos_direitocivil_direitofamilia_dire...,https://questoes.grancursosonline.com.br/aluno...,Questões de Concursos - 1 a 15 - pagina 1,6 - Q3116405\nDireito Civil Direito das Suces...,E,6 - Q3116405,Direito Civil Direito das Sucessões (Art. 1.7...,Ano: 2024,Banca: Fundação Getúlio Vargas - FGV,Prova: FGV - Câmara de São Paulo - Procurador ...,Lofredo faleceu deixando quatro filhas vivas: ...
6,52,granconcursos_direitocivil_direitofamilia_dire...,https://questoes.grancursosonline.com.br/aluno...,Questões de Concursos - 1 a 15 - pagina 1,7 - Q3231293\nDireito Civil Ordem Da Vocação ...,ERRADO,7 - Q3231293,Direito Civil Ordem Da Vocação Hereditária,Ano: 2024,Banca: Instituto Consulplan,Prova: Instituto Consulplan - MPE SC - Promoto...,"João e Maria, amigos de longa data, decidem em..."
7,53,granconcursos_direitocivil_direitofamilia_dire...,https://questoes.grancursosonline.com.br/aluno...,Questões de Concursos - 1 a 15 - pagina 1,8 - Q3116416\nDireito Civil Direito de Famíli...,E,8 - Q3116416,Direito Civil Direito de Família (Art. 1.511 ...,Ano: 2024,Banca: Fundação Getúlio Vargas - FGV,Prova: FGV - Câmara de São Paulo - Procurador ...,"Em março de 2003, Homero e Agatha se casaram p..."
8,54,granconcursos_direitocivil_direitofamilia_dire...,https://questoes.grancursosonline.com.br/aluno...,Questões de Concursos - 1 a 15 - pagina 1,9 - Q3231292\nDireito Civil Direito das Suces...,ERRADO,9 - Q3231292,Direito Civil Direito das Sucessões (Art. 1.7...,Ano: 2024,Banca: Instituto Consulplan,Prova: Instituto Consulplan - MPE SC - Promoto...,"João e Maria, amigos de longa data, decidem em..."
9,55,granconcursos_direitocivil_direitofamilia_dire...,https://questoes.grancursosonline.com.br/aluno...,Questões de Concursos - 1 a 15 - pagina 1,10 - Q3069534\nDireito Civil Sucessão Testame...,A,10 - Q3069534,Direito Civil Sucessão Testamentár

In [101]:
for idx, pergunta in enumerate(concursos_df['pergunta_texto'], 1):
    print('{}.'.format(idx), pergunta, '\n')

1. A sucessão testamentária é um componente importante do planejamento patrimonial e permite que as pessoas tenham controle sobre o destino de seus bens após a morte. É fundamental que um testamento seja elaborado de forma cuidadosa e de acordo com as leis locais para garantir que a vontade do testador seja respeitada e que a sucessão ocorra de maneira adequada. Em relação à sucessão testamentária, analise as afirmativas a seguir.
I. Ao cego só se permite o testamento público, que lhe será lido, em voz alta, duas vezes, uma pelo tabelião ou por seu substituto legal, e a outra por uma das testemunhas, designada pelo testador, fazendo-se de tudo circunstanciada menção no testamento.
II. O testamento cerrado deve ser escrito mecanicamente, desde que seu subscritor numere e autentique, com a sua assinatura, todas as páginas.
III. Em circunstâncias excepcionais declaradas na cédula, o testamento particular de próprio punho e assinado pelo testador, sem testemunhas, deverá ser confirmado, a 

#### Edição manual das perguntas para exemplos



In [104]:
id = 0
concursos_df.loc[id, 'pergunta_texto'] = """A sucessão testamentária é um componente importante do planejamento patrimonial e permite que as pessoas tenham controle sobre o destino de seus bens após a morte. É fundamental que um testamento seja elaborado de forma cuidadosa e de acordo com as leis locais para garantir que a vontade do testador seja respeitada e que a sucessão ocorra de maneira adequada. Em relação à sucessão testamentária, analise as afirmativas a seguir.
I. Ao cego só se permite o testamento público, que lhe será lido, em voz alta, duas vezes, uma pelo tabelião ou por seu substituto legal, e a outra por uma das testemunhas, designada pelo testador, fazendo-se de tudo circunstanciada menção no testamento.
II. O testamento cerrado deve ser escrito mecanicamente, desde que seu subscritor numere e autentique, com a sua assinatura, todas as páginas.
III. Em circunstâncias excepcionais declaradas na cédula, o testamento particular de próprio punho e assinado pelo testador, sem testemunhas, deverá ser confirmado, a critério do juiz.
IV. O testamento particular pode ser escrito em língua estrangeira, contanto que as testemunhas a compreendam.
Está correto o que se afirma apenas em
A) I e II.
B) I e IV.
C) II e IV.
D) III e IV.
"""

concursos_df.loc[id, 'resposta'] = "B"

In [105]:
id = 1
concursos_df.loc[id, 'pergunta_texto'] = """Diego, em razão do falecimento do seu pai Euclides, viúvo, herdou um pequeno sítio localizado em Botucatu, único bem deixado por Euclides. Após a realização de todos os trâmites legais e transferida a propriedade, Diego realizou uma série de benfeitorias no sítio, que estava em situação muito precária, e passou a locá-lo para eventos corporativos, o que se tornou a principal fonte de renda de Diego.
Passados 5 (cinco) anos, Diego é surpreendido com uma citação em ação nulidade de inventário cumulada com petição de herança, promovida por João Carlos, que logrou êxito em provar que também era filho de Euclides, anexando a sentença transitada em julgado da ação de investigação de paternidade, fato que nunca foi e nem poderia ser de conhecimento de Diego.
Diante da situação hipotética narrada, é correto afirmar que Diego, herdeiro aparente e de boa-fé, tem direito a
A) todos os frutos já percebidos, bem como aqueles que forem percebidos antes da prolação eventual sentença de procedência dos pedidos de João Carlos e, também tem direito à indenização das benfeitorias necessárias e úteis.
B) todos os frutos já percebidos, bem como aqueles que forem percebidos antes da prolação eventual sentença de procedência dos pedidos de João Carlos e, também tem direito à indenização das benfeitorias necessárias, úteis e voluptuárias, podendo exercer o direito de retenção.
C) todos os frutos percebidos até a citação, bem como o direito à indenização das benfeitorias, mas só pode exercer o direito de retenção pelo valor das benfeitorias necessárias e úteis.
D) todos os frutos percebidos até a citação, bem como o direito à indenização das benfeitorias necessárias, úteis e voluptuárias, podendo exercer o direito de retenção.
E) todos os frutos já percebidos, bem como aqueles que forem percebidos antes da prolação eventual sentença de procedência dos pedidos de João Carlos e, também tem direito à indenização das benfeitorias, mas só pode exercer o direito de retenção pelo valor das benfeitorias necessárias e úteis.
"""

In [106]:
id = 6
concursos_df.loc[id, 'pergunta_texto'] = """João e Maria, amigos de longa data, decidem empreender juntos e abrem uma empresa de prestação de serviços. No entanto, após alguns meses, a empresa enfrenta dificuldades financeiras e acaba contraindo uma dívida considerável com um fornecedor. Diante disso, surge a discussão sobre a responsabilidade das obrigações de João e Maria em relação a essa dívida. Julgue os itens a seguir.
Se João falecer deixando herdeiros, nenhum destes será obrigado a pagar senão a quota que corresponder ao seu quinhão hereditário, salvo se a obrigação for indivisível; mas todos reunidos poderão ser considerados como um devedor solidário em relação aos demais devedores.
C) Certo
E) Errado
"""
concursos_df.loc[id, 'resposta'] = "E"

In [107]:
concursos_df.iloc[[0,1,6]]

,index,name,link,tema,pergunta,resposta,id,area,ano,banca,prova,pergunta_texto
0,46,granconcursos_direitocivil_direitofamilia_dire...,https://questoes.grancursosonline.com.br/aluno...,Questões de Concursos - 1 a 15 - pagina 1,1 - Q3121863\nDireito Civil Sucessão Testamen...,B,1 - Q3121863,Direito Civil Sucessão Testamentária (Art. 1....,Ano: 2024,Banca: Instituto Consulplan,Prova: Instituto Consulplan - DPE PR - Analist...,A sucessão testamentária é um componente impor...
1,47,granconcursos_direitocivil_direitofamilia_dire...,https://questoes.grancursosonline.com.br/aluno...,Questões de Concursos - 1 a 15 - pagina 1,2 - Q3116412\nDireito Civil Direito das Suces...,C,2 - Q3116412,Direito Civil Direito das Sucessões (Art. 1.7...,Ano: 2024,Banca: Fundação Getúlio Vargas - FGV,Prova: FGV - Câmara de São Paulo - Procurador ...,"Diego, em razão do falecimento do seu pai Eucl..."
6,52,granconcursos_direitocivil_direitofamilia_dire...,https://questoes.grancursosonline.com.br/aluno...,Questões de Concursos - 1 a 15 - pagina 1,7 - Q3231293\nDireito Civil Ordem Da Vocação ...,E,7 - Q3231293,Direito Civil Ordem Da Vocação Hereditária,Ano: 2024,Banca: Instituto Consulplan,Prova: Instituto Consulplan - MPE SC - Promoto...,"João e Maria, amigos de longa data, decidem em..."


In [108]:
concursos_df.loc[7, 'pergunta_texto']

'Em março de 2003, Homero e Agatha se casaram pelo regime da comunhão parcial de bens. Desde o ano passado, Homero teve ciência da infidelidade da esposa ao longo de todo matrimônio. Diante disso, realizou no mês passado exame de DNA de paternidade (DNA Figerprint) de seus filhos, Emanuel, 14 (quatorze) anos, e Sophia, 10 (dez) anos. O resultado reconheceu o vínculo biológico entre Homero e Sophia, porém negou o vínculo entre ele e Emanuel. Destaca-se que desde a concepção Homero sempre foi um pai dedicado, participando cotidianamente da vida de seus filhos, além de ser o principal provedor das despesas da família, o que gerou grande afeto, respeito e admiração dos filhos por ele. Perplexo pelo ocorrido, Homero procura consultoria jurídica desejando o divórcio culposo e a negatória de paternidade com anulação do registro respectivo.\nSobre a hipótese, segundo o sistema jurídico brasileiro, assinale a afirmativa correta.\nA) A infidelidade comprovada é causa legal para o pedido de divór

#### originais

In [112]:
originais_pag1 = """
Diego, em razão do falecimento do seu pai Euclides, viúvo, herdou um pequeno sítio localizado em Botucatu, único bem deixado por Euclides. Após a realização de todos os trâmites legais e transferida a propriedade, Diego realizou uma série de benfeitorias no sítio, que estava em situação muito precária, e passou a locá-lo para eventos corporativos, o que se tornou a principal fonte de renda de Diego.

Passados 5 (cinco) anos, Diego é surpreendido com uma citação em ação nulidade de inventário cumulada com petição de herança, promovida por João Carlos, que logrou êxito em provar que também era filho de Euclides, anexando a sentença transitada em julgado da ação de investigação de paternidade, fato que nunca foi e nem poderia ser de conhecimento de Diego.



Diante da situação hipotética narrada, é correto afirmar que Diego, herdeiro aparente e de boa-fé, tem direito a

A
todos os frutos já percebidos, bem como aqueles que forem percebidos antes da prolação eventual sentença de procedência dos pedidos de João Carlos e, também tem direito à indenização das benfeitorias necessárias e úteis.

B
todos os frutos já percebidos, bem como aqueles que forem percebidos antes da prolação eventual sentença de procedência dos pedidos de João Carlos e, também tem direito à indenização das benfeitorias necessárias, úteis e voluptuárias, podendo exercer o direito de retenção.

C
todos os frutos percebidos até a citação, bem como o direito à indenização das benfeitorias, mas só pode exercer o direito de retenção pelo valor das benfeitorias necessárias e úteis.

D
todos os frutos percebidos até a citação, bem como o direito à indenização das benfeitorias necessárias, úteis e voluptuárias, podendo exercer o direito de retenção.

E
todos os frutos já percebidos, bem como aqueles que forem percebidos antes da prolação eventual sentença de procedência dos pedidos de João Carlos e, também tem direito à indenização das benfeitorias, mas só pode exercer o direito de retenção pelo valor das benfeitorias necessárias e úteis.


Responder**

A sucessão testamentária é um componente importante do planejamento patrimonial e permite que as pessoas tenham controle sobre o destino de seus bens após a morte. É fundamental que um testamento seja elaborado de forma cuidadosa e de acordo com as leis locais para garantir que a vontade do testador seja respeitada e que a sucessão ocorra de maneira adequada. Em relação à sucessão testamentária, analise as afirmativas a seguir.


I. Ao cego só se permite o testamento público, que lhe será lido, em voz alta, duas vezes, uma pelo tabelião ou por seu substituto legal, e a outra por uma das testemunhas, designada pelo testador, fazendo-se de tudo circunstanciada menção no testamento.

II. O testamento cerrado deve ser escrito mecanicamente, desde que seu subscritor numere e autentique, com a sua assinatura, todas as páginas.

III. Em circunstâncias excepcionais declaradas na cédula, o testamento particular de próprio punho e assinado pelo testador, sem testemunhas, deverá ser confirmado, a critério do juiz.

IV. O testamento particular pode ser escrito em língua estrangeira, contanto que as testemunhas a compreendam.


Está correto o que se afirma apenas em

A
I e II.

B
I e IV.

C
II e IV.

D
III e IV.


Responder**

Matheus, filho de um milionário, aos dezesseis anos, decide escrever, de próprio punho, seu testamento. No testamento, escrito em língua francesa, Matheus decide dispor da totalidade de seus bens. Passados cinco meses, mediante escrito particular seu, datado e assinado, Matheus decide fazer disposições especiais sobre o seu enterro. Além disso, decide alterar seu testamento, dispondo apenas de seus bens imóveis. Passados dois meses, decide novamente alterar seu testamento, dispondo novamente da totalidade de seus bens. Tanto a elaboração do testamento quanto suas retificações foram lidas e assinadas por Matheus na presença de três testemunhas, que o subscreveram. Diante da situação hipotética, assinale a alternativa correta.

A
O codicilo só é válido se não houver testamento; havendo testamento, as disposições de última vontade deverão nele ser incluídas.

B
O testamento não é válido, sendo necessária a maioridade para sua validade ou a participação do Ministério Público, enquanto menor.

C
O testamento é ato personalíssimo, podendo ser alterado a qualquer tempo mediante manifestação favorável do Ministério Público.

D
Extingue-se em três anos o direito de impugnar a validade do testamento, contado o prazo de sua abertura.

E
O testamento pode ser escrito em língua estrangeira desde que as testemunhas a compreendam.


Responder**

Mariana, solteira, não convivente em união estável, relativamente incapaz em razão da idade, sem filhos e sem ascendentes vivos, elaborou, de próprio punho, testamento particular, o qual foi lido por ela na presença de três testemunhas que o subscreveram, dispondo sobre a destinação post mortem de todos os bens integrantes de seu patrimônio.

Pelo instrumento, Mariana determinou que o imóvel no qual reside, recebido por herança de seus pais, seria transferido à Clara, sua amiga de infância, gravado com cláusula de inalienabiliedade e incomunicabilidade. Dispôs ainda que as ações de sua titularidade seriam destinadas à Associação Patinhas e a constituição de uma fundação de defesa dos interesses dos animais, sem determinar como seria a divisão de quotas. Destinou um imóvel rural, também recebido por herança de seus pais, ao filho que Pedro, seu primo, viesse a ter. Por fim, destinou a biblioteca de livros jurídicos, incluindo obras raras, que pertenceu ao seu avô já falecido, para Túlio, estudante de direito e filho de Joana, uma das testemunhas que subscreveu o testamento.



Considerando a situação hipotética narrada e as normas jurídicas que regem a sucessão testamentária, assinale a opção correta.

A
O testamento de Mariana, em razão da sua incapacidade civil no momento da sua elaboração, independentemente da sua capacidade no momento da abertura da sucessão, é anulável, assim como a deixa em favor de Túlio é nula, por ser ele presumidamente pessoa interposta a não legitimado a suceder.

B
A incapacidade relativa de Mariana no momento da elaboração do testamento afeta a validade do ato, mas a deixa em favor de Clara constitui legado que, em razão da abusividade dos gravames impostos, será tido como puro e simples.

C
A incapacidade relativa de Mariana no momento da elaboração do testamento não afeta a validade do ato, assim como é válida a deixa em favor do filho que Pedro vier a ter, mas a deixa em favor de Túlio é nula.

D
A incapacidade relativa de Mariana no momento da elaboração do testamento não afeta a validade do ato, mas a destinação do imóvel rural ao filho que Pedro vier a ter será tida por não escrita, na eventualidade de o contemplado não ser coexistente ao autor da herança.

E
A incapacidade relativa de Mariana no momento da elaboração do testamento não afeta a validade do ato, mas a destinação do imóvel rural ao filho que Pedro vier a ter é inválida; a deixa das ações para a Associação Patinhas e para a constituição de uma fundação de defesa dos interesses dos animais é válida.


Responder**

Lofredo faleceu deixando quatro filhas vivas: Petrúcia, Rossana, Tertúlia e Vanusa. As duas primeiras são fruto de seu primeiro casamento, com Helena. As duas últimas são provenientes de seu segundo casamento, com Afrodite, com quem estava casado, na vigência da sociedade conjugal, no momento de seu falecimento, segundo o regime da comunhão universal de bens. Lofredo deixou um testamento público, elaborado na forma da lei e sem vícios, no qual destinou 30% de seu patrimônio a Merida, sua sobrinha. Porém, à época da abertura da sucessão de Lofredo, Merida já havia falecido, deixando duas filhas vivas (Desiré e Efigênia).



Diante deste fato, assinale a afirmativa correta.

A
São herdeiras de Lofredo as suas quatro filhas, a esposa atual (Afrodite) e sua sobrinha Merida.

B
Na concorrência sucessória entre Afrodite e as filhas de Lofredo, deve-se reservar o quinhão mínimo de um quatro da herança ao cônjuge sobrevivente.

C
Com relação ao quinhão hereditário destinado pela via testamentária, trasmitir-se-á, por representação, à Desiré e Efigênia.

D
Considerando o regime de bens, Afrodite herdará o valor correspondente à metade da herança destinada aos herdeiros legítimos.

E
O quinhão destinado à Rossana equivalerá a 17,5% do patrimônio deixado por Lofredo.


Responder**

Rogério é casado com Cláudio e ambos são pais de João. Em razão de uma oportunidade de trabalho, Rogéri o passa a semana em Cabo Frio e volta para o Rio de J aneiro aos finais de semana para ficar com o marido e filho. Dois anos após o início das viagens de Rogério, ele passou a se relacionar, durante as semanas com Vitor, sendo publicamente reconhecido como seu companheiro em Cabo Frio e continuou casado com Cláudio, com o qual passava os finais de semana. A situação perdurou por oito anos, até que, em um acidente de carro, Rogério veio a falecer.



Diante da situação hipotética e de acordo com o atual entendimento do Supremo Tribunal Federal, é correto afirmar que

A
é possível a configuração da união estável entre Rogério e Vitor considerando que a relação entre eles era pública, duradoura e contínua.

B
seria possível o reconhecimento da união estável apenas se se tratasse de relacionamento heteroafetivo.

C
seria possível a configuração da união estável entre Rogério e Vitor, caso Rogério estivesse separado de fato de Cláudio.

D
não seria possível a configuração da união estável entre Rogério e Vitor, mas a este caberia metade da pensão por morte do INSS por ser daquele dependente.

E
seria possível a configuração da união estável entre Rogério e Vitor, caso Rogério convivesse em união estável com Cláudio.


Responder**

João e Maria, amigos de longa data, decidem empreender juntos e abrem uma empresa de prestação de serviços. No entanto, após alguns meses, a empresa enfrenta dificuldades financeiras e acaba contraindo uma dívida considerável com um fornecedor. Diante disso, surge a discussão sobre a responsabilidade das obrigações de João e Maria em relação a essa dívida. Julgue os itens a seguir.
Se João falecer deixando herdeiros, nenhum destes será obrigado a pagar senão a quota que corresponder ao seu quinhão hereditário, salvo se a obrigação for indivisível; mas todos reunidos poderão ser considerados como um devedor solidário em relação aos demais devedores.

C
Certo

E
Errado


Responder**

Em março de 2003, Homero e Agatha se casaram pelo regime da comunhão parcial de bens. Desde o ano passado, Homero teve ciência da infidelidade da esposa ao longo de todo matrimônio. Diante disso, realizou no mês passado exame de DNA de paternidade (DNA Figerprint) de seus filhos, Emanuel, 14 (quatorze) anos, e Sophia, 10 (dez) anos. O resultado reconheceu o vínculo biológico entre Homero e Sophia, porém negou o vínculo entre ele e Emanuel. Destaca-se que desde a concepção Homero sempre foi um pai dedicado, participando cotidianamente da vida de seus filhos, além de ser o principal provedor das despesas da família, o que gerou grande afeto, respeito e admiração dos filhos por ele. Perplexo pelo ocorrido, Homero procura consultoria jurídica desejando o divórcio culposo e a negatória de paternidade com anulação do registro respectivo.



Sobre a hipótese, segundo o sistema jurídico brasileiro, assinale a afirmativa correta.

A
A infidelidade comprovada é causa legal para o pedido de divórcio que levará a perda do direito ao nome e aos alimentos, além do dever de indenizar.

B
Homero poderá promover a ação de divórcio culposo com o pedido de exclusão da adúltera a partilha dos bens.

C
A jurisprudência do Supremo Tribunal Federal aboliu do ordenamento jurídico brasileiro o divórcio culposo, contudo para a propositura da ação negatória basta o exame negativo de paternidade.

D
Para propositura da ação negatória de paternidade, não basta o exame de DNA, que reconhece a inexistência do vínculo biológico, é preciso a confissão expressa da genitora, além do consentimento expresso do adolescente.

E
Com base na jurisprudência consolidada do Superior Tribunal de Justiça, para ação negatória de paternidade cumulada com pedido de anulação de registro, é preciso demonstrar a inexistência de relação socioafetiva entre pai e filho.


Responder**

João e Maria, amigos de longa data, decidem empreender juntos e abrem uma empresa de prestação de serviços. No entanto, após alguns meses, a empresa enfrenta dificuldades financeiras e acaba contraindo uma dívida considerável com um fornecedor. Diante disso, surge a discussão sobre a responsabilidade das obrigações de João e Maria em relação a essa dívida. Julgue os itens a seguir.

Se um dos credores solidários falecer deixando herdeiros, cada herdeiro terá direito a exigir e receber a quota do crédito que corresponder à parte da herança que lhe foi destinada, independentemente da obrigação ser ou não indivisível.

C
Certo

E
Errado


Responder**

Fernando, viúvo, faleceu em Brasília, seu último domicílio, deixando três filhos: Ana, a primogênita, tem domicílio em Natal; Jorge, o filho do meio, tem domicílio em Aracaju; e Luís, o caçula e indicado pelos irmãos para exercer a inventariança do pai, tem domicílio no Rio de Janeiro.



Segundo a legislação vigente, para a elaboração do inventário e partilha extrajudiciais, aos filhos de Jorge é facultado celebrar o ato

A
em Brasília.

B
no Rio de Janeiro.

C
em Natal.

D
em Aracaju.

E
em qualquer lugar do país.


Responder**

O Banco XYZ é credor de Diana, que não possui outros bens além de um carro e valores correntes necessários para sua sobrevivência, uma vez que não lograra sucesso na vida empresarial.

Determinado dia, Diana é surpreendida com a informação de que Judith, irmã bilateral de seu pai, com quem pouco tinha contato, havia falecido e lhe havia deixado herança a ser compartilhada com outros 3 (três) primos, todos filhos de outro irmão bilateral do pai de Diana.

Por conta da dívida com o Banco XYZ, Diana decide renunciar à herança, para que não seja integralmente consumida pela instituição financeira. Em pesquisa de localização de bens de Diana, o Banco XYZ tomou ciência da renúncia feita em ofício de notas, bem como identificar o inventário judicial ainda em curso de Judith.



Diante destes fatos, o Banco XYZ

A
poderá aceitar a herança e receber metade do monte deixado por Judith.

B
nada poderá fazer, senão continuar buscando bens de Diana.

C
deverá mover ação pauliana contra os primos de Diana, não lhe cabendo se habilitar no inventário.

D
poderá aceitar a herança e receber um quarto do monte deixado por Judith.

E
deverá se habilitar no inventário dos bens de Judith para receber seu crédito.


Responder**

No que se refere ao Direito Das Sucessões, segundo o Código Civil, é correto afirmar:

A
O menor de dezoito anos não pode fazer o seu testamento.

B
O testamento é ato personalíssimo, podendo ser mudado a qualquer tempo.

C
A sucessão dá-se só por disposição de última vontade.

D
São herdeiros necessários os descendentes, os ascendentes, o colateral e o cônjuge.


Responder**

Marcelo tem deficiência visual em alto grau, que lhe impossibilita ler. Diante disso, para que ele possa realizar um testamento válido, salvo se a higidez da manifestação do falecido for completamente assegurada, será necessário adotar, por força de lei:

A
necessariamente a forma pública. Nesse caso, o teor do documento deve ser lido para Marcelo em voz alta pelo tabelião e por uma das testemunhas;

B
necessariamente a forma pública. Nesse caso, o teor do documento deve ser lido para Marcelo em voz alta por ambas as testemunhas perante o tabelião;

C
a forma pública ou cerrada. Se cerrado, o documento deve ser escrito por uma das duas pessoas que testemunharão a entrega ao tabelião;

D
a forma pública ou cerrada. Se cerrado, o documento deve ser escrito por pessoa de confiança a rogo de Marcelo e lido por uma das duas pessoas que testemunharão a entrega ao tabelião;

E
a forma pública, cerrada ou particular. Se cerrado ou particular, o documento deve ser escrito por pessoa de confiança a rogo de Marcelo e lido por uma das três pessoas que testemunharam a sua elaboração.


Responder**

Acerca das disposições do Código Civil sobre o testamento, é INCORRETO afirmar que:

A
Se o testador não souber, ou não puder assinar, o tabelião ou seu substituto legal assim o declarará, assinando, neste caso, pelo testador, e, a seu rogo, uma das testemunhas instrumentárias.

B
Lavrado o testamento, deve ser lido em voz alta pelo tabelião ao testador e a quatro testemunhas, a um só tempo; ou pelo testador, se o quiser, na presença destas e do oficial.

C
Ao cego só se permite o testamento público, que lhe será lido, em voz alta, duas vezes, uma pelo tabelião ou por seu substituto legal, e a outra por uma das testemunhas, designada pelo testador, fazendo-se de tudo circunstanciada menção no testamento.

D
O indivíduo inteiramente surdo, sabendo ler, lerá o seu testamento, e, se não o souber, designará quem o leia em seu lugar, presentes as testemunhas.


Responder**

Assinale a alternativa correta. Quanto à sucessão legítima, dispõe o Código Civil que,

A
na falta de descendentes, são chamados à sucessão os ascendentes, sem a concorrência com o cônjuge sobrevivente, por já fazer jus à meação.

B
se não houver colaterais, até o quinto grau, será chamado a suceder o cônjuge sobrevivente.

C
não sobrevivendo cônjuge, ou companheiro, nem parente algum sucessível, ou tendo eles renunciado à herança, ela será devolvida ao Município ou ao Distrito Federal, se localizada nas respectivas circunscrições, ou à União, quando situada em território federal.

D
em falta de descendentes e ascendentes, será deferida a sucessão por inteiro aos colaterais.

E
na sucessão do falecido, os irmãos unilaterais herdarão, na condição de igualdade, com os irmãos bilaterais.


Responder**

João, nascido em Brasília – DF, viveu toda a sua vida em Penha – SC e morreu em Florianópolis – SC. Quando ele morreu, Ana, sua esposa, estava grávida.

Como João morreu em um hospital localizado em Florianópolis – SC, este deverá ser o lugar da abertura de sua sucessão.

C
Certo

E
Errado


Responder**

João, nascido em Brasília – DF, viveu toda a sua vida em Penha – SC e morreu em Florianópolis – SC. Quando ele morreu, Ana, sua esposa, estava grávida.

Se João tivesse indicado, mediante testamento, a filha de um amigo à sucessão, o ato só seria válido se ela já tivesse nascido quando da lavratura do testamento.

C
Certo

E
Errado


Responder**

No direito de família, os alimentos

A
não podem ser fixados em favor dos pais contra os filhos, em razão do princípio da hereditariedade.

B
poderão ser cobrados dos avós da criança, caso o genitor ou a genitora não estejam em condições de pagar alimentos.

C
devem ser fixados na proporção das necessidades do reclamante, sem considerar os recursos da pessoa obrigada.

D
não são devidos aos filhos havidos fora do casamento.

E
podem ser renunciados pela criança em relação ao pai, quando representada por sua genitora.


Responder**

Podem caracterizar a impossibilidade da comunhão de vida a ocorrência de algum dos seguintes motivos, EXCETO:

A
Condenação por crime infamante.

B
Tentativa de morte.

C
Abandono involuntário do lar conjugal, durante um ano contínuo.

D
Adultério.


Responder**

A respeito da herança jacente, assinale a opção correta, considerando a legislação vigente acerca do assunto e o entendimento do STJ.

A
Até a declaração de vacância, corre o prazo para que o imóvel possa ser usucapido pelo particular que o detém.

B
O estado-membro é o sucessor dos bens jacentes.

C
É vedado ao juiz do domicílio do autor da herança determinar, de ofício, a abertura do procedimento de arrecadação da herança jacente.

D
Logo que aberta a sucessão, os bens que compõem a herança jacente passam a integrar o patrimônio do ente público.

E
A jacência, por pressupor incerteza quanto à existência dos herdeiros, tem como consequência necessária a declaração de vacância.
"""

In [124]:
originais_pag2 = """
O parentesco entre as pessoas naturais poderá ser civil, natural ou por afinidade. Por isso, é correto afirmar que

A
irmãos bilaterais são parentes na linha colateral ou transversal em primeiro grau; os unilaterais, em segundo grau.

B
por afinidade, o parentesco na linha colateral existe até o quarto grau.

C
os cônjuges são parentes entre si pelo casamento, constituindo vínculo de parentesco civil.

D
pela adoção, é constituído parentesco civil entre pais e filho na linha reta em primeiro grau.

///

Inês, viúva, sem herdeiros necessários, faz um testamento por meio de escritura pública deixando todo o seu patrimônio para Laura, sua melhor amiga, com o ônus de ser tutora e cuidadora de seu animal doméstico, Frajola.



Sobre a hipótese, com base no ordenamento jurídico brasileiro, assinale a afirmativa correta.

A
O testamento está sujeito a um encargo, não havendo suspensão da aquisição nem do exercício do direito, salvo quando expressamente imposto no testamento, pela testadora, como condição suspensiva.

B
O encargo de tutela e cuidado do animal assemelha-se ao termo inicial, sendo considerado, por conseguinte, um evento futuro e certo.

C
O caso é vedado pelo ordenamento jurídico brasileiro, por não haver previsão legal a respeito de encargo.

D
O testamento sujeita-se a um encargo, sendo o motivo determinante da liberalidade, sendo considerado inexistente, caso seja impossível seu cumprimento.

E
O testamento sujeita-se à transferência do patrimônio a um termo suspensivo, que vem a ser cuidar do animal; assim, em primeiro lugar, Laura deverá cuidar do animal e, após a morte natural dele, a propriedade tornar-se-á plena.

///

A cessão de direitos hereditários é ato permitido aos herdeiros, em conformidade com o disposto no Código Civil. A respeito do assunto, leia as assertivas abaixo.



I. O coerdeiro não poderá ceder onerosamente a sua quota hereditária à pessoa estranha à sucessão se outro coerdeiro a quiser adquirir, pelo mesmo valor.

II. É ineficaz a cessão pelo coerdeiro de qualquer bem do acervo hereditário considerado singularmente.

III. Realizada a cessão de direitos hereditários, os direitos conferidos ao herdeiro cedente em consequência de substituição ou de direito de acrescer, presumem-se abrangidos pela cessão feita anteriormente.



Considerando as assertivas acima, assinale a alternativa correta:

A
Apenas estão corretas as assertivas II e III.

B
Apenas estão corretas as assertivas I e II.

C
Apenas a assertiva III está correta.

D
Estão corretas as assertivas I, II, III.

///

Ainda sobre o direito de Sucessão é correto afirmar que:

A
Somente é reconhecido direito sucessório ao cônjuge sobrevivente se, ao tempo da morte do outro, não estavam separados judicialmente, nem separados de fato há mais de 2 anos, salvo prova, neste caso, de que essa convivência se tornara impossível sem culpa do sobrevivente.

B
Somente é reconhecido direito sucessório ao cônjuge sobrevivente se, ao tempo da morte do outro, não estavam separados judicialmente, nem separados de fato há mais de 4 anos, salvo prova, neste caso, de que essa convivência se tornara impossível sem culpa do sobrevivente.

C
Somente é reconhecido direito sucessório ao cônjuge sobrevivente se, ao tempo da morte do outro, não estavam separados judicialmente, nem separados de fato há mais de 8 anos, salvo prova, neste caso, de que essa convivência se tornara impossível sem culpa do sobrevivente.

D
Somente é reconhecido direito sucessório ao cônjuge sobrevivente se, ao tempo da morte do outro, não estavam separados judicialmente, nem separados de fato há mais de 1 ano, salvo prova, neste caso, de que essa convivência se tornara impossível sem culpa do sobrevivente.

///

Ana faleceu deixando dois filhos, Pedro e Paulo, e 3 netos: dois filhos de Pedro e um filho de Paulo. Pedro, no entanto, praticou crime contra a honra do companheiro de Ana pouco antes do falecimento da autora da herança. Nesse caso, é correto afirmar, conforme o Código Civil:

A
Pedro, por ser herdeiro necessário, não pode ser excluído da sucessão por indignidade.

B
Pedro será automaticamente excluído por indignidade, no próprio inventário, seja judicial ou extrajudicial.

C
Pedro poderá ser excluído por indignidade, tendo o Ministério Público legitimidade para demandar a exclusão.

D
A pretensão de exclusão e de Pedro por indignidade prescreve em 5 (cinco) anos, contados da abertura da sucessão.

E
Caso Pedro seja excluído da sucessão por indignidade, seus herdeiros sucederão, como se Pedro fosse morto antes da abertura da sucessão de Ana.

///

Conforme previsto no Código Civil, disposição testamentária em favor de pessoa não legitimada a suceder será considerada

A
inexistente.

B
anulável.

C
nula.

D
válida, se não ultrapassar cinquenta por cento do monte.

E
ineficaz.

///

Bento faleceu na última segunda-feira deixando um vasto patrimônio. Ele era casado com Glória, pelo regime de separação convencional de bens, e deixou dois filhos vivos, Cosme e José.



Ocorre que Justina, também filha de Bento e Glória, que falecera dois anos antes de Bento, tinha dois filhos absolutamente incapazes, Pedro (5 anos de idade) e Valentina (3 anos), que se encontram vivos na presente data.



Com base no ordenamento jurídico vigente, assinale a opção que apresenta a correta partilha de bens de Bento.



Obs.: para a solução do caso, Bento nunca realizou testamento.

A
O patrimônio de Bento será dividido em parte iguais entre Cosme e José.

B
Glória terá direito a metade dos bens de Bento, na qualidade de meeira, e o restante divido em parte iguais entre Cosme e José.

C
O patrimônio de Bento será dividido em quatro partes. Glória, Cosme e José herdarão, cada um, a quarta parte do patrimônio, sendo que a outra quarta parte será dividida entre Pedro e Valentina, que herdarão por representação.

D
O patrimônio de Bento será dividido em parte iguais entre Glória, Cosme e José.

E
O patrimônio de Bento será dividido em três partes. Cosme e José herdarão, cada um, a terça parte do patrimônio, sendo que a outra terça parte será dividida entre Pedro e Valentina, que herdarão por representação.

///

O testamento é ato personalíssimo e irrevogável.

C
Certo

E
Errado

///

É proibido o testamento conjuntivo, seja simultâneo, recíproco ou correspectivo.

C
Certo

E
Errado

///

Ao cego só se permite o testamento público, que lhe será lido, em voz alta, duas vezes pelo tabelião, fazendo-se de tudo circunstanciada a menção no testamento.

C
Certo

E
Errado

///

Mariana falece em outubro de 2020 sem deixar testamento e também sem herdeiros necessários. De seus quatro irmãos germanos, Marcos, Mário, Mirtes e Maitê, Mário, pai de Augusto, por ter muito boa situação financeira, renuncia à herança. Mirtes, mãe de Jéssica, foi declarada indigna em relação à sucessão de Mariana por sentença transitada em julgado proferida junto ao Juízo Orfanológico. Por fim, Maitê e seu único filho Igor faleceram em acidente de carro no ano de 2018, tendo Igor deixado filha única, Ana, sobrinha-neta de Mariana.



Serão chamados à sucessão de Mariana:

A
Marcos, Jéssica e Ana;

B
Marcos e Jéssica;

C
Marcos e Augusto;

D
Marcos e Ana;

E
Marcos, Augusto e Ana.

///

De acordo com o Código Civil, são excluídos da sucessão os herdeiros ou legatários que



I tiverem sido autores, coautores ou partícipes de homicídio doloso, ou tentativa deste, contra a pessoa de cuja sucessão se tratar seu cônjuge, companheiro, ascendente ou descendente.

II tiverem acusado caluniosamente em juízo o autor da herança ou incorrerem em crime contra a sua honra, ou de seu cônjuge ou de seu companheiro.

III por violência ou meios fraudulentos, inibirem ou obstarem o autor da herança de dispor livremente de seus bens por ato de última vontade.



Assinale a opção correta.

A
Apenas o item II está certo.

B
Apenas o item III está certo.

C
Apenas os itens I e II estão certos.

D
Apenas os itens I e III estão certos.

E
Todos os itens estão certos.

///

Maria, com 17 anos, tramou e executou o assassinato de seus pais, para que pudesse ficar com a respectiva herança, avaliada em dezenas de milhões de reais. Pretendia, com isso, prover uma vida de luxos à sua filha, Mariazinha, o que vinha sendo negado pelos avós.



Nesse caso, é correto afirmar que:

A
se cometido por um maior de idade, o caso seria de deserdação, pela prática de homicídio doloso contra os autores da herança; no entanto, tratando-se de ato infracional análogo cometido por menor de idade, não é possível a interpretação extensiva para excluir Maria da sucessão, diante da taxatividade do rol do Art. 1.962 do Código Civil;

B
ainda que praticado por menor de idade, o ato infracional análogo ao homicídio doloso cometido contra os autores da herança justifica a deserdação em uma leitura teleológica e sistemática compatível com a taxatividade do rol do Art. 1.962 do Código Civil; nesse caso, a herança passará a Mariazinha, como se a mãe fosse pré-morta;

C
ainda que praticado por menor de idade, o ato infracional análogo ao homicídio doloso cometido contra os autores da herança justifica o reconhecimento da indignidade em uma leitura teleológica e sistemática compatível com a taxatividade do rol do Art. 1.814 do Código Civil; nesse caso, a herança passará a Mariazinha, como se a mãe fosse pré-morta;

D
se cometido por um maior de idade, o caso seria de indignidade, pela prática de homicídio doloso contra os autores da herança; no entanto, tratando-se de ato infracional análogo cometido por menor de idade, não é possível a interpretação extensiva para excluir Maria da sucessão, diante da taxatividade do rol do Art. 1.814 do Código Civil;

E
ainda que praticado por menor de idade, o ato infracional análogo ao homicídio doloso cometido contra os autores da herança justifica o reconhecimento da indignidade em uma leitura teleológica e sistemática compatível com a taxatividade do rol do Art. 1.962 do Código Civil; nesse caso, com a exclusão de sua mãe da sucessão, Mariazinha não poderá receber nada, até porque isso representaria um aproveitamento da própria torpeza.

///

José, viúvo de Maria, falecida em 2019, com quem teve 2 filhos, faleceu em 2022. Em testamento cerrado, reconheceu a paternidade de uma filha fora do casamento; a ela, no mesmo testamento, deixou sua parte disponível, instituindo-a na qualidade de herdeira testamentária. Nesse caso, como será partilhada a herança de José?

A
Os filhos de José com Maria partilharão a parte indisponível, por serem herdeiros legítimos, e a filha a disponível, por ser herdeira testamentária.

B
A filha de José, por ser irmã unilateral de seus filhos com Maria, na parte indisponível, herdará metade da cota parte de seus irmãos.

C
Os filhos de José com Maria, por serem herdeiros necessários, partilharão todo o patrimônio de José, em partes iguais, tanto a parte disponível como a indisponível.

D
A parte indisponível da herança de José será partilhada entre seus filhos e sua filha, em partes iguais; a filha será a herdeira testamentária na parte disponível.

///

O direito de sucessão hereditária será legítimo ou testamentário. Assim sendo, é correto afirmar que

A
possuem vocação hereditária à sucessão legítima pessoa natural e nascituro; à testamentária, também prole eventual de determinada pessoa – desde que viva quando da abertura da sucessão –, e pessoa jurídica, constituída ou a ser constituída sob a forma de fundação.

B
o sobrinho do falecido é seu herdeiro legítimo, não necessário, e herda em igualdade de condições com eventual tio do falecido, também herdeiro legítimo, não necessário.

C
o direito de representação aplica-se à sucessão legítima, não à testamentária, na linha reta, e na colateral aos filhos de irmãos do falecido, quando com irmãos deste concorrerem.

D
a pessoa com absoluta ou relativa incapacidade civil está impossibilitada de dispor de seus bens por testamento.

///

“Um foragido da justiça foi preso pela Polícia Rodoviária Federal (PRF), em uma abordagem na BR 101, em Eunápolis, no Extremo Sul da Bahia. O flagrante aconteceu na madrugada de hoje, 14 de janeiro. Policiais estavam fiscalizando na rodovia, quando abordaram um ônibus de viagem que seguia do Rio de Janeiro com destino a cidade de Água Branca, no Alagoas.

Durante a fiscalização, em consultas ao sistema de segurança, a equipe da PRF verificou que um dos passageiros, um homem de 45 anos, possuía mandado de prisão por pensão alimentícia, expedido em março de 2022 pela Comarca de Porto Seguro (BA). Ele foi preso e conduzido para a Delegacia de Polícia Civil”.



Disponível em: https://www.gov.br/prf/pt-br/noticias/estaduais/bahia/2023/janeiro/homem-e-preso-em-eunapolis-ba-por-naopagamento-de-pensao-alimenticia



Sobre alimentos no Direito de Família e a jurisprudência dos tribunais superiores, analise as assertivas e identifique com V as verdadeiras e com F as falsas.



( ) Julgada procedente a investigação de paternidade, os alimentos são devidos a partir da citação.

( ) O cancelamento de pensão alimentícia de filho que atingiu a maioridade está sujeito à decisão judicial, mediante contraditório, ainda que nos próprios autos.

( ) Os efeitos da sentença, que reduz, majora ou exonera o alimentante do pagamento, retroagem à data da citação, vedadas a compensação e a repetibilidade.

( ) O débito alimentar que autoriza a prisão civil do alimentante é o que compreende as três prestações anteriores à intimação e as que vencerem no curso do processo.

( ) A obrigação alimentar dos avós tem natureza complementar e subsidiária, somente se configurando no caso de impossibilidade total ou parcial de seu cumprimento pelos pais.



A alternativa que contém a sequência correta, de cima para baixo, é

A
V V V F V

B
V F F F V

C
V V F V V

D
F F V F V

E
F F F F V

///

Cada um dos itens a seguir apresenta uma situação hipotética seguida de uma assertiva a ser julgada conforme as disposições do Código Civil e da jurisprudência do STJ em relação à proteção da pessoa dos filhos em situações de multiparentalidade.



I O pai biológico de Maria faleceu quando ela tinha apenas doze anos de idade. Dois anos depois, a mãe de Maria passou a viver em união estável com João. Desde então, João tomou para si o exercício da função paterna na vida de Maria, situação plenamente aceita por ela. Por essa razão, João e Maria decidiram tornar jurídica a situação fática então existente, para ser reconhecida a paternidade socioafetiva dele mediante sua inclusão no registro civil dela, sem exclusão do pai biológico falecido. Nessa situação hipotética, reconhecida a multiparentalidade em razão da ligação afetiva entre enteada e padrasto, Maria terá direitos patrimoniais e sucessórios em relação tanto ao pai falecido quanto a João.

II Regina namorava publicamente Adão e outros rapazes quando engravidou. Dois meses depois do nascimento de Felipe, fruto dessa gravidez, Adão o registrou e passou a tratá-lo publicamente como filho. Todavia, com dúvidas acerca da paternidade, Adão fez, extrajudicialmente, um exame de DNA e constatou que Felipe não era seu filho biológico. Nessa situação hipotética, a divergência entre a paternidade biológica e a declarada no registro de nascimento é suficiente para que Adão possa pleitear judicialmente a anulação do ato registral, mesmo configurada a paternidade socioafetiva.

III Daniel e Jonas convivem em união estável homoafetiva e resolveram ter um filho. Procuraram, então, uma clínica de fertilização na companhia de Marta, irmã de Jonas, para um programa de inseminação artificial. Daniel e Marta se submeteram ao ciclo de reprodução assistida, dando origem a Letícia. Marta foi somente a chamada barriga solidária. Nessa situação hipotética, o registro civil de Letícia deverá ser realizado pelo cartório, independentemente de prévia autorização judicial.

IV Quando Eva se casou com Ivo, já era mãe de Elias, fruto de um relacionamento anterior. Embora Elias seja filho biológico e registral de outro homem, perante a sociedade, o trabalho, os amigos e a escola, Ivo sempre o apresenta como seu filho, sem qualquer distinção. Nessa situação hipotética, depois do falecimento de Ivo, Elias poderá obter judicialmente o reconhecimento de Ivo como seu pai socioafetivo, incluindo-o no seu registro civil, sem a exclusão do pai biológico.



Estão certos apenas os itens

A
I e II.

B
I e III.

C
II e IV.

D
I, III, e IV.

E
II, III e IV.

///

A legítima dos herdeiros necessários não poderá ser incluída no testamento.

C
Certo

E
Errado

///

Renato, nascido em 20/2/1985, ajuizou ação de investigação de paternidade post mortem cumulada com petição de herança em desfavor de Mariana e de Juliana, filhas herdeiras de Manoel, indicado como suposto pai biológico do autor. Na petição inicial, protocolada em 15/12/2022, Renato demonstrou que, no dia 5/2/2010, ocorreu o falecimento de Manoel e informou que, apenas posteriormente, optou por tomar medida jurídica para que houvesse o reconhecimento da paternidade e a restituição de herança.



Nessa situação hipotética, a pretensão de Renato referente à petição de herança

A
não está prescrita, porque ainda não se consumou o prazo prescricional, que deve ser contado da abertura da sucessão.

B
pode ser exercida, por ser imprescritível.

C
somente pode ser apresentada após a prolação de julgamento definitivo em investigação de paternidade, em cuja data se inicia a contagem do prazo prescricional para petição de herança.

D
somente pode ser apresentada após a prolação de decisão de procedência em investigação de paternidade, em cuja data se inicia a contagem do prazo prescricional para petição de herança.

E
está prescrita, porque já se consumou o prazo prescricional, que deve ser contado da abertura da sucessão.

///

João, nascido em Brasília – DF, viveu toda a sua vida em Penha – SC e morreu em Florianópolis – SC. Quando ele morreu, Ana, sua esposa, estava grávida.



Considerando essa situação hipotética e o disposto no Código Civil sobre o direito das sucessões, julgue os itens subsequentes.

Nessa situação, como Ana estava grávida quando da morte de João, o nascituro tem legitimidade para suceder.

C
Certo

E
Errado


"""

In [113]:
originais_pag1 = originais_pag1.split('Responder**')
len(originais_pag1)

20

In [125]:
originais_pag2 = originais_pag2.split('///')
len(originais_pag2)

20

### Edição automática

In [114]:
class KEEP_CONCURSO_STYLE_QUESTION_EDITOR_JSON(BaseModel):
    pergunta_original: str = Field(description="pergunta original recebida")
    pergunta_editada: str = Field(description="pergunta editada")

output_parser = JsonOutputParser(pydantic_object=KEEP_CONCURSO_STYLE_QUESTION_EDITOR_JSON)

In [115]:
prompt = """
Você foi contratado para organizar um banco de perguntas de concurso público.
Para te ajudar seu chefe forneceu três exemplos de edição.

Exemplo 1:
Pergunta original: {original_exemplo_1}
Pergunta editada: {editada_exemplo_1}

Exemplo 2:
Pergunta original: {original_exemplo_2}
Pergunta editada: {editada_exemplo_2}

Exemplo 3:
Pergunta original: {original_exemplo_3}
Pergunta editada: {editada_exemplo_3}

É crucial para sua promoção que você não altere os textos originais, apenas sua formatação.

Agora faça o mesmo para a pergunta a seguir,:

Pergunta: {pergunta}

Instruções de saída: {format_instructions}
"""

prompt_template = PromptTemplate(
    input_variables=["original_exemplo_1", "editada_exemplo_1",
                     "original_exemplo_2", "editada_exemplo_2",
                     "original_exemplo_3", "editada_exemplo_3",
                     "pergunta"],
    template=prompt,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt_template | llama3_70b_temp0 | output_parser

In [120]:
{"original_exemplo_1": originais_pag1[0],
"editada_exemplo_1": concursos_df.loc[1, 'pergunta_texto'],
"original_exemplo_2": originais_pag1[1],
"editada_exemplo_2": concursos_df.loc[0, 'pergunta_texto'],
"original_exemplo_3": originais_pag1[6],
"editada_exemplo_3": concursos_df.loc[6, 'pergunta_texto']}

{'original_exemplo_1': '\nDiego, em razão do falecimento do seu pai Euclides, viúvo, herdou um pequeno sítio localizado em Botucatu, único bem deixado por Euclides. Após a realização de todos os trâmites legais e transferida a propriedade, Diego realizou uma série de benfeitorias no sítio, que estava em situação muito precária, e passou a locá-lo para eventos corporativos, o que se tornou a principal fonte de renda de Diego.\n\nPassados 5 (cinco) anos, Diego é surpreendido com uma citação em ação nulidade de inventário cumulada com petição de herança, promovida por João Carlos, que logrou êxito em provar que também era filho de Euclides, anexando a sentença transitada em julgado da ação de investigação de paternidade, fato que nunca foi e nem poderia ser de conhecimento de Diego.\n\n\n\nDiante da situação hipotética narrada, é correto afirmar que Diego, herdeiro aparente e de boa-fé, tem direito a\n\nA\ntodos os frutos já percebidos, bem como aqueles que forem percebidos antes da prola

In [121]:
def call_keep_question_editor(pergunta, df, originais):
    out = chain.invoke({
        "original_exemplo_1": originais[0],
        "editada_exemplo_1": df.loc[1, 'pergunta_texto'],
        "original_exemplo_2": originais[1],
        "editada_exemplo_2": df.loc[0, 'pergunta_texto'],
        "original_exemplo_3": originais[6],
        "editada_exemplo_3": df.loc[6, 'pergunta_texto'],
        "pergunta": pergunta
    })
    return out

# test
call_keep_question_editor(originais_pag1[2], concursos_df, originais_pag1)

{'pergunta_original': 'Matheus, filho de um milionário, aos dezesseis anos, decide escrever, de próprio punho, seu testamento. No testamento, escrito em língua francesa, Matheus decide dispor da totalidade de seus bens. Passados cinco meses, mediante escrito particular seu, datado e assinado, Matheus decide fazer disposições especiais sobre o seu enterro. Além disso, decide alterar seu testamento, dispondo apenas de seus bens imóveis. Passados dois meses, decide novamente alterar seu testamento, dispondo novamente da totalidade de seus bens. Tanto a elaboração do testamento quanto suas retificações foram lidas e assinadas por Matheus na presença de três testemunhas, que o subscreveram. Diante da situação hipotética, assinale a alternativa correta.\nA) O codicilo só é válido se não houver testamento; havendo testamento, as disposições de última vontade deverão nele ser incluídas.\nB) O testamento não é válido, sendo necessária a maioridade para sua validade ou a participação do Ministér

#### Loop

In [122]:
output_list = []
for i, pergunta in tqdm(enumerate(originais_pag1)):
    try:
        ascore_json = call_keep_question_editor(pergunta, concursos_df, originais_pag1)
        output_list.append(ascore_json)
        print(ascore_json)
    except Exception as e:
        print(f"Erro na iteração {i}: {e}")
        output_list.append('ERROR')
        continue

question_df = pd.DataFrame(output_list)
question_df.to_csv("Perguntas_DATASET2_SAME.csv", index=False)

1it [00:04,  4.10s/it]

{'pergunta_original': 'Diego, em razão do falecimento do seu pai Euclides, viúvo, herdou um pequeno sítio localizado em Botucatu, único bem deixado por Euclides. Após a realização de todos os trâmites legais e transferida a propriedade, Diego realizou uma série de benfeitorias no sítio, que estava em situação muito precária, e passou a locá-lo para eventos corporativos, o que se tornou a principal fonte de renda de Diego. Passados 5 (cinco) anos, Diego é surpreendido com uma citação em ação nulidade de inventário cumulada com petição de herança, promovida por João Carlos, que logrou êxito em provar que também era filho de Euclides, anexando a sentença transitada em julgado da ação de investigação de paternidade, fato que nunca foi e nem poderia ser de conhecimento de Diego. Diante da situação hipotética narrada, é correto afirmar que Diego, herdeiro aparente e de boa-fé, tem direito a A) todos os frutos já percebidos, bem como aqueles que forem percebidos antes da prolação eventual sen

2it [00:19, 10.48s/it]

{'pergunta_original': 'A sucessão testamentária é um componente importante do planejamento patrimonial e permite que as pessoas tenham controle sobre o destino de seus bens após a morte. É fundamental que um testamento seja elaborado de forma cuidadosa e de acordo com as leis locais para garantir que a vontade do testador seja respeitada e que a sucessão ocorra de maneira adequada. Em relação à sucessão testamentária, analise as afirmativas a seguir.\nI. Ao cego só se permite o testamento público, que lhe será lido, em voz alta, duas vezes, uma pelo tabelião ou por seu substituto legal, e a outra por uma das testemunhas, designada pelo testador, fazendo-se de tudo circunstanciada menção no testamento.\nII. O testamento cerrado deve ser escrito mecanicamente, desde que seu subscritor numere e autentique, com a sua assinatura, todas as páginas.\nIII. Em circunstâncias excepcionais declaradas na cédula, o testamento particular de próprio punho e assinado pelo testador, sem testemunhas, de

3it [00:56, 22.72s/it]

...

{'pergunta_original': 'No direito de família, os alimentos\nA) não podem ser fixados em favor dos pais contra os filhos, em razão do princípio da hereditariedade.\nB) poderão ser cobrados dos avós da criança, caso o genitor ou a genitora não estejam em condições de pagar alimentos.\nC) devem ser fixados na proporção das necessidades do reclamante, sem considerar os recursos da pessoa obrigada.\nD) não são devidos aos filhos havidos fora do casamento.\nE) podem ser renunciados pela criança em relação ao pai, quando representada por sua genitora.', 'pergunta_editada': 'No direito de família, os alimentos\nA) não podem ser fixados em favor dos pais contra os filhos, em razão do princípio da hereditariedade.\nB) poderão ser cobrados dos avós da criança, caso o genitor ou a genitora não estejam em condições de pagar alimentos.\nC) devem ser fixados na proporção das necessidades do reclamante, sem considerar os recursos da pessoa obrigada.\nD) não são devidos aos filhos havidos fora do casam

19it [10:02, 30.58s/it]

{'pergunta_original': 'Podem caracterizar a impossibilidade da comunhão de vida a ocorrência de algum dos seguintes motivos, EXCETO: A) Condenação por crime infamante. B) Tentativa de morte. C) Abandono involuntário do lar conjugal, durante um ano contínuo. D) Adultério.', 'pergunta_editada': 'Podem caracterizar a impossibilidade da comunhão de vida a ocorrência de algum dos seguintes motivos, EXCETO: A) Condenação por crime infamante. B) Tentativa de morte. C) Abandono involuntário do lar conjugal, durante um ano contínuo. D) Adultério.'}


20it [10:32, 31.63s/it]

{'pergunta_original': 'A respeito da herança jacente, assinale a opção correta, considerando a legislação vigente acerca do assunto e o entendimento do STJ.\n\nA) Até a declaração de vacância, corre o prazo para que o imóvel possa ser usucapido pelo particular que o detém.\n\nB) O estado-membro é o sucessor dos bens jacentes.\n\nC) É vedado ao juiz do domicílio do autor da herança determinar, de ofício, a abertura do procedimento de arrecadação da herança jacente.\n\nD) Logo que aberta a sucessão, os bens que compõem a herança jacente passam a integrar o patrimônio do ente público.\n\nE) A jacência, por pressupor incerteza quanto à existência dos herdeiros, tem como consequência necessária a declaração de vacância.', 'pergunta_editada': 'A respeito da herança jacente, assinale a opção correta, considerando a legislação vigente acerca do assunto e o entendimento do STJ.\nA) Até a declaração de vacância, corre o prazo para que o imóvel possa ser usucapido pelo particular que o detém.\nB)

In [129]:
for i, pergunta in tqdm(enumerate(originais_pag2)):
    try:
        ascore_json = call_keep_question_editor(pergunta, concursos_df, originais_pag1)
        output_list.append(ascore_json)
        print(ascore_json)
    except Exception as e:
        print(f"Erro na iteração {i}: {e}")
        output_list.append('ERROR')
        continue

1it [00:01,  1.72s/it]

{'pergunta_original': 'O parentesco entre as pessoas naturais poderá ser civil, natural ou por afinidade. Por isso, é correto afirmar que\nA) irmãos bilaterais são parentes na linha colateral ou transversal em primeiro grau; os unilaterais, em segundo grau.\nB) por afinidade, o parentesco na linha colateral existe até o quarto grau.\nC) os cônjuges são parentes entre si pelo casamento, constituindo vínculo de parentesco civil.\nD) pela adoção, é constituído parentesco civil entre pais e filho na linha reta em primeiro grau.', 'pergunta_editada': 'O parentesco entre as pessoas naturais poderá ser civil, natural ou por afinidade. Por isso, é correto afirmar que\nA) irmãos bilaterais são parentes na linha colateral ou transversal em primeiro grau; os unilaterais, em segundo grau.\nB) por afinidade, o parentesco na linha colateral existe até o quarto grau.\nC) os cônjuges são parentes entre si pelo casamento, constituindo vínculo de parentesco civil.\nD) pela adoção, é constituído parentes

2it [00:04,  2.38s/it]

{'pergunta_original': 'Inês, viúva, sem herdeiros necessários, faz um testamento por meio de escritura pública deixando todo o seu patrimônio para Laura, sua melhor amiga, com o ônus de ser tutora e cuidadora de seu animal doméstico, Frajola. Sobre a hipótese, com base no ordenamento jurídico brasileiro, assinale a afirmativa correta. A) O testamento está sujeito a um encargo, não havendo suspensão da aquisição nem do exercício do direito, salvo quando expressamente imposto no testamento, pela testadora, como condição suspensiva. B) O encargo de tutela e cuidado do animal assemelha-se ao termo inicial, sendo considerado, por conseguinte, um evento futuro e certo. C) O caso é vedado pelo ordenamento jurídico brasileiro, por não haver previsão legal a respeito de encargo. D) O testamento sujeita-se a um encargo, sendo o motivo determinante da liberalidade, sendo considerado inexistente, caso seja impossível seu cumprimento. E) O testamento sujeita-se à transferência do patrimônio a um te

3it [00:35, 15.63s/it]

...

{'pergunta_original': 'A legítima dos herdeiros necessários não poderá ser incluída no testamento. C) Certo E) Errado', 'pergunta_editada': 'A legítima dos herdeiros necessários não poderá ser incluída no testamento. C) Certo E) Errado'}


19it [09:41, 36.01s/it]

{'pergunta_original': 'Renato, nascido em 20/2/1985, ajuizou ação de investigação de paternidade post mortem cumulada com petição de herança em desfavor de Mariana e de Juliana, filhas herdeiras de Manoel, indicado como suposto pai biológico do autor. Na petição inicial, protocolada em 15/12/2022, Renato demonstrou que, no dia 5/2/2010, ocorreu o falecimento de Manoel e informou que, apenas posteriormente, optou por tomar medida jurídica para que houvesse o reconhecimento da paternidade e a restituição de herança. Nessa situação hipotética, a pretensão de Renato referente à petição de herança', 'pergunta_editada': 'Renato, nascido em 20/2/1985, ajuizou ação de investigação de paternidade post mortem cumulada com petição de herança em desfavor de Mariana e de Juliana, filhas herdeiras de Manoel, indicado como suposto pai biológico do autor. Na petição inicial, protocolada em 15/12/2022, Renato demonstrou que, no dia 5/2/2010, ocorreu o falecimento de Manoel e informou que, apenas poster

20it [10:13, 30.68s/it]

{'pergunta_original': 'João, nascido em Brasília – DF, viveu toda a sua vida em Penha – SC e morreu em Florianópolis – SC. Quando ele morreu, Ana, sua esposa, estava grávida. Considerando essa situação hipotética e o disposto no Código Civil sobre o direito das sucessões, julgue os itens subsequentes. Nessa situação, como Ana estava grávida quando da morte de João, o nascituro tem legitimidade para suceder. C) Certo E) Errado', 'pergunta_editada': 'João, nascido em Brasília – DF, viveu toda a sua vida em Penha – SC e morreu em Florianópolis – SC. Quando ele morreu, Ana, sua esposa, estava grávida. Considerando essa situação hipotética e o disposto no Código Civil sobre o direito das sucessões, julgue os itens subsequentes. Nessa situação, como Ana estava grávida quando da morte de João, o nascituro tem legitimidade para suceder. C) Certo E) Errado'}


In [54]:
question_df = pd.DataFrame(output_list)
question_df.to_csv("Perguntas_DATASET2_SAME.csv", index=False)
question_df

,pergunta_original,pergunta_editada
0,Pergunta: A sucessão testamentária é um compon...,Pergunta: A sucessão testamentária é um compon...
1,"Diego, em razão do falecimento do seu pai Eucl...","Diego, em razão do falecimento do seu pai Eucl..."
2,"Matheus, filho de um milionário, aos dezesseis...","Matheus, filho de um milionário, aos dezesseis..."
3,"Mariana, solteira, não convivente em união est...","Mariana, solteira, não convivente em união est..."
4,Rogério é casado com Cláudio e ambos são pais ...,Rogério é casado com Cláudio e ambos são pais ...
5,Lofredo faleceu deixando quatro filhas vivas: ...,Lofredo faleceu deixando quatro filhas vivas: ...
6,"João e Maria, amigos de longa data, decidem em...","João e Maria, amigos de longa data, decidem em..."
7,"Em março de 2003, Homero e Agatha se casaram p...","Em março de 2003, Homero e Agatha se casaram p..."
8,"João e Maria, amigos de longa data, decidem em...","João e Maria, amigos de longa data, decidem em..."
9,"Fernando, viúvo, faleceu em Brasília, seu últi...","Fernando, viúvo, faleceu em Brasília, seu últi..."


In [83]:
# pergunta_nova = """No direito de família, os alimentos

# A
# não podem ser fixados em favor dos pais contra os filhos, em razão do princípio da hereditariedade.

# B
# poderão ser cobrados dos avós da criança, caso o genitor ou a genitora não estejam em condições de pagar alimentos.

# C
# devem ser fixados na proporção das necessidades do reclamante, sem considerar os recursos da pessoa obrigada.

# D
# não são devidos aos filhos havidos fora do casamento.

# E
# podem ser renunciados pela criança em relação ao pai, quando representada por sua genitora."""

# ascore_json = call_keep_question_editor(pergunta_nova, concursos_df, originais)
# output_list[17] = ascore_json

In [85]:
# pergunta_nova = """Podem caracterizar a impossibilidade da comunhão de vida a ocorrência de algum dos seguintes motivos, EXCETO:

# A
# Condenação por crime infamante.

# B
# Tentativa de morte.

# C
# Abandono involuntário do lar conjugal, durante um ano contínuo.

# D
# Adultério."""

# ascore_json = call_keep_question_editor(pergunta_nova, concursos_df, originais)
# output_list[18] = ascore_json

## Headers

In [135]:
headers = """
1
Q3116412
Direito Civil  Direito das Sucessões (Art. 1.784 ao 2.027)
Ano: 2024
Banca: Fundação Getúlio Vargas - FGV
Prova: FGV - Câmara de São Paulo - Procurador Legislativo - 2024


2
Q3121863
Direito Civil  Sucessão Testamentária (Art. 1.857 ao 1.990)
Ano: 2024
Banca: Instituto Consulplan
Prova: Instituto Consulplan - DPE PR - Analista - Área Direito - 2024


3
Q3251846
Direito Civil  Sucessão Testamentária (Art. 1.857 ao 1.990)
Ano: 2024
Banca: Fundação para o Vestibular da Universidade Estadual Paulista - VUNESP
Prova: VUNESP - MPE RO - Promotor de Justiça Substituto - 2024


4
Q3158359
Direito Civil  Sucessão Testamentária (Art. 1.857 ao 1.990)
Ano: 2024
Banca: Fundação Getúlio Vargas - FGV
Prova: FGV - MPE GO - Promotor de Justiça Substituto - 2024


5
Q3116405
Direito Civil  Direito das Sucessões (Art. 1.784 ao 2.027)
Ano: 2024
Banca: Fundação Getúlio Vargas - FGV
Prova: FGV - Câmara de São Paulo - Procurador Legislativo - 2024


6
Q3192677
Direito Civil  Direito de Família (Art. 1.511 ao 1.783)
Ano: 2024
Banca: Fundação para o Vestibular da Universidade Estadual Paulista - VUNESP
Prova: VUNESP - MPE RJ - Promotor de Justiça Substituto - 2024


7
Q3231292
Direito Civil  Direito das Sucessões (Art. 1.784 ao 2.027)
Ano: 2024
Banca: Instituto Consulplan
Prova: Instituto Consulplan - MPE SC - Promotor de Justiça Substituto - Manhã - 2024


8
Q3116416
Direito Civil  Direito de Família (Art. 1.511 ao 1.783)
Ano: 2024
Banca: Fundação Getúlio Vargas - FGV
Prova: FGV - Câmara de São Paulo - Procurador Legislativo - 2024


9
Q3231293
Direito Civil  Direito das Sucessões (Art. 1.784 ao 2.027)
Ano: 2024
Banca: Instituto Consulplan
Prova: Instituto Consulplan - MPE SC - Promotor de Justiça Substituto - Manhã - 2024


10
Q3071800
Direito Civil  Direito das Sucessões (Art. 1.784 ao 2.027)
Ano: 2023
Banca: Fundação Getúlio Vargas - FGV
Prova: FGV - Câmara dos Deputados - Analista Legislativo - Consultor Legislativo - Área: II - 2023


11
Q3071802
Direito Civil  Direito das Sucessões (Art. 1.784 ao 2.027)
Ano: 2023
Banca: Fundação Getúlio Vargas - FGV
Prova: FGV - Câmara dos Deputados - Analista Legislativo - Consultor Legislativo - Área: II - 2023


12
Q2694663
Direito Civil  Noções Gerais de Sucessão Testamentária
Ano: 2023
Banca: MS Concursos
Prova: MS Concursos - Prefeitura de Patrocínio - Advogado - 2023


13
Q3069534
Direito Civil  Sucessão Testamentária (Art. 1.857 ao 1.990)
Ano: 2023
Banca: Fundação Getúlio Vargas - FGV
Prova: FGV - TJ PR - Juiz Substituto - 2023


14
Q3025338
Direito Civil  Sucessão Testamentária (Art. 1.857 ao 1.990)
Ano: 2023
Banca: Instituto de Estudos Superiores do Extremo Sul - IESES
Prova: IESES - TJ AM - Titular de Serviços Notariais e Registrais - Remoção - 2023


15
Q3057367
Direito Civil  Sucessão Legítima (Art. 1.829 ao 1.856)
Ano: 2023
Banca: Fundação para o Vestibular da Universidade Estadual Paulista - VUNESP
Prova: VUNESP - Prefeitura de São Paulo - Auditor Fiscal Tributário Municipal - Área Gestão Tributária - 2023


16
Q2802282
Direito Civil  Direito das Sucessões (Art. 1.784 ao 2.027)
Ano: 2023
Banca: Centro de Seleção e de Promoção de Eventos UnB - CESPE CEBRASPE
Prova: CESPE/CEBRASPE - MPE SC - Promotor de Justiça Substituto - Manhã - 2023


17
Q2802283
Direito Civil  Direito das Sucessões (Art. 1.784 ao 2.027)
Ano: 2023
Banca: Centro de Seleção e de Promoção de Eventos UnB - CESPE CEBRASPE
Prova: CESPE/CEBRASPE - MPE SC - Promotor de Justiça Substituto - Manhã - 2023


18
Q2857311
Direito Civil  Direito de Família (Art. 1.511 ao 1.783)
Ano: 2023
Banca: Fundação Carlos Chagas - FCC
Prova: FCC - MPE PB - Técnico Ministerial - Área Sem Especialidade - 2023


19
Q3025444
Direito Civil  Direito de Família (Art. 1.511 ao 1.783)
Ano: 2023
Banca: Instituto de Estudos Superiores do Extremo Sul - IESES
Prova: IESES - TJ AM - Titular de Serviços Notariais e Registrais - Remoção - 2023


20
Q2912796
Direito Civil  Teoria Geral da Sucessão - Sucessão em geral (Art. 1.784 ao 1.828)
Ano: 2023
Banca: Centro de Seleção e de Promoção de Eventos UnB - CESPE CEBRASPE
Prova: CESPE/CEBRASPE - PGM - Procurador Municipal - 2023


21
Q2695567
Direito Civil  Direito de Família (Art. 1.511 ao 1.783)
Ano: 2023
Banca: Instituto AOCP
Prova: Instituto AOCP - MPE RR - Promotor de Justiça Substituto - 2023


22
Q2675041
Direito Civil  Sucessão Testamentária (Art. 1.857 ao 1.990)
Ano: 2023
Banca: Fundação Getúlio Vargas - FGV
Prova: FGV - SEFAZ MG - Auditor Fiscal da Receita Estadual - Área: Tributação - 2023


23
Q3034664
Direito Civil  Direito das Sucessões (Art. 1.784 ao 2.027)
Ano: 2023
Banca: Instituto de Estudos Superiores do Extremo Sul - IESES
Prova: IESES - TJ AM - Titular de Serviços Notariais e Registrais - Provimento - 2023


24
Q2899089
Direito Civil  Sucessão do Companheiro e do Cônjuge
Ano: 2023
Banca: Funatec
Prova: Funatec - Prefeitura de Palmeirante - Assistente Jurídico - 2023


25
Q2800095
Direito Civil  Exclusão da Sucessão
Ano: 2023
Banca: Centro de Extensão, Treinamento e Aperfeiçoamento Profissional Ltda - CETAP
Prova: CETAP - IGEPREV - Técnico Previdenciário - 2023


26
Q2939598
Direito Civil  Sucessão Testamentária (Art. 1.857 ao 1.990)
Ano: 2023
Banca: Centro de Seleção e de Promoção de Eventos UnB - CESPE CEBRASPE
Prova: CESPE/CEBRASPE - MPE BA - Promotor de Justiça Substituto - 2023


27
Q2675023
Direito Civil  Direito das Sucessões (Art. 1.784 ao 2.027)
Ano: 2023
Banca: Fundação Getúlio Vargas - FGV
Prova: FGV - SEFAZ MG - Auditor Fiscal da Receita Estadual - Área: Tributação - 2023


28
Q2732922
Direito Civil  Sucessão Testamentária (Art. 1.857 ao 1.990)
Ano: 2023
Banca: Instituto Quadrix - Quadrix
Prova: Quadrix - CREF 3 - Advogado - 2023


29
Q2732924
Direito Civil  Sucessão Testamentária (Art. 1.857 ao 1.990)
Ano: 2023
Banca: Instituto Quadrix - Quadrix
Prova: Quadrix - CREF 3 - Advogado - 2023


30
Q2732926
Direito Civil  Sucessão Testamentária (Art. 1.857 ao 1.990)
Ano: 2023
Banca: Instituto Quadrix - Quadrix
Prova: Quadrix - CREF 3 - Advogado - 2023


31
Q3028625
Direito Civil  Direito das Sucessões (Art. 1.784 ao 2.027)
Ano: 2023
Banca: Fundação Getúlio Vargas - FGV
Prova: FGV - DPE RJ - Defensor Público de Classe Inicial - 2023


32
Q2754155
Direito Civil  Exclusão da Sucessão
Ano: 2023
Banca: Centro de Seleção e de Promoção de Eventos UnB - CESPE CEBRASPE
Prova: CESPE/CEBRASPE - MPE PA - Promotor Substituto - 2023


33
Q2837617
Direito Civil  Direito das Sucessões (Art. 1.784 ao 2.027)
Ano: 2023
Banca: Fundação Getúlio Vargas - FGV
Prova: FGV - TJ MS - Juiz Substituto - 2023


34
Q2695574
Direito Civil  Sucessão Testamentária (Art. 1.857 ao 1.990)
Ano: 2023
Banca: Instituto AOCP
Prova: Instituto AOCP - MPE RR - Promotor de Justiça Substituto - 2023


35
Q2695576
Direito Civil  Direito das Sucessões (Art. 1.784 ao 2.027)
Ano: 2023
Banca: Instituto AOCP
Prova: Instituto AOCP - MPE RR - Promotor de Justiça Substituto - 2023


36
Q2718839
Direito Civil  Direito de Família (Art. 1.511 ao 1.783)
Ano: 2023
Banca: Fundação de Apoio à Educação e Desenvolvimento Tecnológico da Bahia - CEFETBAHIA
Prova: CEFETBAHIA - DPE BA - Analista Técnico - Área Direito - 2023


37
Q2719195
Direito Civil  Direito de Família (Art. 1.511 ao 1.783)
Ano: 2023
Banca: Centro de Seleção e de Promoção de Eventos UnB - CESPE CEBRASPE
Prova: CESPE/CEBRASPE - TJDFT - Juiz Substituto - 2023


38
Q2732919
Direito Civil  Sucessão Testamentária (Art. 1.857 ao 1.990)
Ano: 2023
Banca: Instituto Quadrix - Quadrix
Prova: Quadrix - CREF 3 - Advogado - 2023


39
Q2723957
Direito Civil  Direito das Sucessões (Art. 1.784 ao 2.027)
Ano: 2023
Banca: Centro de Seleção e de Promoção de Eventos UnB - CESPE CEBRASPE
Prova: CESPE/CEBRASPE - MPE AM - Promotor de Justiça Substituto - 2023


40
Q2802286
Direito Civil  Direito das Sucessões (Art. 1.784 ao 2.027)
Ano: 2023
Banca: Centro de Seleção e de Promoção de Eventos UnB - CESPE CEBRASPE
Prova: CESPE/CEBRASPE - MPE SC - Promotor de Justiça Substituto - Manhã - 2023
"""

In [65]:
# question_df = pd.DataFrame(output_list)
# question_df.loc[0, 'pergunta_editada'].split(':')

['Pergunta',
 ' A sucessão testamentária é um componente importante do planejamento patrimonial e permite que as pessoas tenham controle sobre o destino de seus bens após a morte. É fundamental que um testamento seja elaborado de forma cuidadosa e de acordo com as leis locais para garantir que a vontade do testador seja respeitada e que a sucessão ocorra de maneira adequada. Em relação à sucessão testamentária, analise as afirmativas a seguir. I. Ao cego só se permite o testamento público, que lhe será lido, em voz alta, duas vezes, uma pelo tabelião ou por seu substituto legal, e a outra por uma das testemunhas, designada pelo testador, fazendo-se de tudo circunstanciada menção no testamento. II. O testamento cerrado deve ser escrito mecanicamente, desde que seu subscritor numere e autentique, com a sua assinatura, todas as páginas. III. Em circunstâncias excepcionais declaradas na cédula, o testamento particular de próprio punho e assinado pelo testador, sem testemunhas, deverá ser c

## Respostas

In [136]:
A = 'A'
B = 'B'
C = 'C'
D = 'D'
E = 'E'
respostas = [C, B, E, C, E, C, E, E, E, E, D, B, A, B, C, E, E, B, C, A,
             D, A, B, A, E, C, C, E, C, E, B, E, C, D, A, A, D, C, E, C]
len(respostas)

40

## Comentários

In [90]:
comentarios_pag1 = """
CC

Art. 1.826. O possuidor da herança está obrigado à restituição dos bens do acervo, fixando-se-lhe a responsabilidade segundo a sua posse, observado o disposto nos arts. 1.214 a 1.222.

Parágrafo único. A partir da citação, a responsabilidade do possuidor se há de aferir pelas regras concernentes à posse de má-fé e à mora.



Art. 1.214. O possuidor de boa-fé tem direito, enquanto ela durar, aos frutos percebidos.

Parágrafo único. Os frutos pendentes ao tempo em que cessar a boa-fé devem ser restituídos, depois de deduzidas as despesas da produção e custeio; devem ser também restituídos os frutos colhidos com antecipação.



Art. 1.219. O possuidor de boa-fé tem direito à indenização das benfeitorias necessárias e úteis, bem como, quanto às voluptuárias, se não lhe forem pagas, a levantá-las, quando o puder sem detrimento da coisa, e poderá exercer o direito de retenção pelo valor das benfeitorias necessárias e úteis.
Art. 1.973. Sobrevindo descendente sucessível ao testador, que não o tinha ou não o conhecia quando testou, rompe-se o testamento em todas as suas disposições, se esse descendente sobreviver ao testador.

Art. 1.974. Rompe-se também o testamento feito na ignorância de existirem outros herdeiros necessários.

STF já decidiu que o rompimento do testamento só ocorre se provado que o testador não tinha conhecimento do novo herdeiro necessário.

Art. 1.975. Não se rompe o testamento, se o testador dispuser da sua metade, não contemplando os herdeiros necessários de cuja existência saiba, ou quando os exclua dessa parte.

Obs: Nesse caso, o testamento é modificado quanto à reservada legítima. (Fonte: https://www.jusbrasil.com.br/artigos/testamentos-invalidados-e-ineficazes-revogacao-rompimento-caducidade-anulabilidade-e-nulidade/759470990)



Na hipótese de nulidade de inventário, hipótese da questão, o prazo prescricional é de 10 (dez) anos, salvo melhor juízo. Isso porque, por dedução, nulidade de inventário não encontra previsão nas hipóteses do art. 206. Logo:

Art. 205. A prescrição ocorre em dez anos, quando a lei não lhe haja fixado prazo menor.
O possuidor de boa-fé tem direito à indenização das benfeitorias necessárias e úteis, bem como os frutos já percebidos.

///

I -

Art. 1.867. Ao cego só se permite o testamento público, que lhe será lido, em voz alta, duas vezes, uma pelo tabelião ou por seu substituto legal, e a outra por uma das testemunhas, designada pelo testador, fazendo-se de tudo circunstanciada menção no testamento.





II -

Art. 1.868. O testamento escrito pelo testador, ou por outra pessoa, a seu rogo, e por aquele assinado, será válido se aprovado pelo tabelião ou seu substituto legal, observadas as seguintes formalidades:

I - que o testador o entregue ao tabelião em presença de duas testemunhas;



II - que o testador declare que aquele é o seu testamento e quer que seja aprovado;



III - que o tabelião lavre, desde logo, o auto de aprovação, na presença de duas testemunhas, e o leia, em seguida, ao testador e testemunhas;



IV - que o auto de aprovação seja assinado pelo tabelião, pelas testemunhas e pelo testador.



Parágrafo único. O testamento cerrado pode ser escrito mecanicamente, desde que seu subscritor numere e autentique, com a sua assinatura, todas as páginas.



III -

Art. 1.879. Em circunstâncias excepcionais declaradas na cédula, o testamento particular de próprio punho e assinado pelo testador, sem testemunhas, poderá ser confirmado, a critério do juiz.





IV -

Art. 1.880. O testamento particular pode ser escrito em língua estrangeira, contanto que as testemunhas a compreendam.

///

A resposta da questão está no Art. 1.880. O testamento particular pode ser escrito em língua estrangeira, contanto que as testemunhas a compreendam.
Art. 1.860. Além dos incapazes, não podem testar os que, no ato de fazê-lo, não tiverem pleno discernimento.

Parágrafo único. Podem testar os maiores de dezesseis anos.

Art. 1.861. A incapacidade superveniente do testador não invalida o testamento, nem o testamento do incapaz se valida com a superveniência da capacidade.
Art. 1.858. O testamento é ato personalíssimo, podendo ser mudado a qualquer tempo.



Art. 1.859. Extingue-se em cinco anos o direito de impugnar a validade do testamento, contado o prazo da data do seu registro.



Art. 1.860. Além dos incapazes, não podem testar os que, no ato de fazê-lo, não tiverem pleno discernimento. Parágrafo único. Podem testar os maiores de dezesseis anos.



Art. 1.876. O testamento particular pode ser escrito de próprio punho ou mediante processo mecânico.

§ 1 o Se escrito de próprio punho, são requisitos essenciais à sua validade seja lido e assinado por quem o escreveu, na presença de pelo menos três testemunhas, que o devem subscrever.



Art. 1.880. O testamento particular pode ser escrito em língua estrangeira, contanto que as testemunhas a compreendam.

///

Art. 1.860. Além dos incapazes, não podem testar os que, no ato de fazê-lo, não tiverem pleno discernimento.

Parágrafo único. Podem testar os maiores de dezesseis anos.



Art. 1.799. Na sucessão testamentária podem ainda ser chamados a suceder:

I - os filhos, ainda não concebidos, de pessoas indicadas pelo testador, desde que vivas estas ao abrir-se a sucessão;



Art. 1.900. É nula a disposição:

V - que favoreça as pessoas a que se referem os arts. 1.801 e 1.802.



Art. 1.801. Não podem ser nomeados herdeiros nem legatários:

I - a pessoa que, a rogo, escreveu o testamento, nem o seu cônjuge ou companheiro, ou os seus ascendentes e irmãos;

II - as testemunhas do testamento;

III - o concubino do testador casado, salvo se este, sem culpa sua, estiver separado de fato do cônjuge há mais de cinco anos;

IV - o tabelião, civil ou militar, ou o comandante ou escrivão, perante quem se fizer, assim como o que fizer ou aprovar o testamento.

Art. 1.802. São nulas as disposições testamentárias em favor de pessoas não legitimadas a suceder, ainda quando simuladas sob a forma de contrato oneroso, ou feitas mediante interposta pessoa.

Parágrafo único. Presumem-se pessoas interpostas os ascendentes, os descendentes, os irmãos e o cônjuge ou companheiro do não legitimado a suceder.

Art. 1.860. Além dos incapazes, não podem testar os que, no ato de fazê-lo, não tiverem pleno discernimento.

Parágrafo único. Podem testar os maiores de dezesseis anos.

Art. 1.861. A incapacidade superveniente do testador não invalida o testamento, nem o testamento do incapaz se valida com a superveniência da capacidade.

///

Essa questão não tem gabarito certo. Explico.

Não pode ser a “A” porque, à época do falecimento, Lofredo era casado no regime de comunhão universal, impossibilitando que a esposa, Afrodite, herdasse. Nesse caso, na forma do art. 1.829, I, do CC, a esposa só ficaria com 50% dos bens, em razão do regime adotado, não concorrendo com os herdeiros.

Também não é a “B” pelo mesmo motivo supracitado. Além disso, ainda que a esposa fosse considerada herdeira, haveria a vedação disposta no art. 1.832 do CC, já que os 04 filhos não são comuns.

Não é a C porque a beneficiária do testamento é pré-morta, não havendo indicação de substituição (inteligência do art. 1.939, 1.941, 1.943 e 1.944 do CC). Como se sabe, na sucessão testamentária, não há que se falar em direito de representação (o valor disposto retorna para o monte).

Não é a D pelos mesmos motivos da A.

Por fim, não tem como ser a “E”, já que, se Rossana recebesse 17,5% do patrimônio, por consequência, as outras filhas também receberiam o mesmo quinhão, totalizando, desse modo, 70% do patrimônio de Lofredo. Registre-se que 50% do patrimônio pertence a Afrodite, pelo regime de bens, sobrando tão somente 50% para fins de partilha entre as filhas. Outra coisa, ainda que se considerasse 100% dos 50% que caberia às herdeiras, não tem como Rossana ficar com 17,5%. A conta não fecharia, salvo se desprezasse os 30% da sobrinha, MAS não é o caso (e aqui reside o erro), já que a parcela dela retorna para o monte-mor.

O adequado seria 25% para cada filha, se for calcular 100% da herança, ou 12,5%, se incidir nos 50%.

Não ha resposta correta.

O regime de comunhão universal de bens exclui a condição de herdeira da esposa .

Não há direito de representação em sucessão testamentária. os 30% voltam ao montante.

Os bens deixados serão repartidos entre as 4 filhas

Essa alternativa não faz sentido nenhum, a esposa fica com 50% por ser casada na universal, os outros 50% seriam dispostos no testamento, mas como a sobrinha faleceu, não existe sucessão testamentária, logo, os 30% retornam ao montante restante, totalizando mais 50% para serem partilhados entre as filhas, sendo assim, 50%/4=12,5%. Não entendo como que uma filha ganharia 17,5%, sendo que para chegar nesse resultado teria de considerar que os 30% não retornam ao montante devido ao falecimento da sobrinha, restando 70% que seriam divididos entre as filhas (70%/4=17,5%), só que nesse caso, a cônjuge não receberia nada, que também está errado.
O cálculo está errado, -30%=70% a ser dividido entre as 4 filhas+1esposa=5 pessoas é 70%/5 e não por 4.
Fiz contas e contas e não consegui bater com esse gabarito kkkkk, se algum ser milagroso conseguir por favor explique.

///

Gabarito C.





A preexistência de casamento ou de união estável de um dos conviventes, ressalvada a exceção do artigo 1.723, § 1º, do Código Civil, impede o reconhecimento de novo vínculo referente ao mesmo período, inclusive para fins previdenciários, em virtude da consagração do dever de fidelidade e da monogamia pelo ordenamento jurídico-constitucional brasileiro. STF. Plenário. RE 1045273, Rel. Min. Alexandre de Moraes, julgado em 18/12/2020 (Repercussão Geral – Tema 529) (Info 1003).

Art. 1.723. É reconhecida como entidade familiar a união estável entre o homem e a mulher, configurada na convivência pública, contínua e duradoura e estabelecida com o objetivo de constituição de família.

ADI 4.277 E ADPF 132 - STF se utilizou da técnica de interpretação conforme à Constituição a fim de "excluir do dispositivo em causa qualquer significado que impeça o reconhecimento da união contínua, pública e duradoura entre pessoas do mesmo sexo como família. Reconhecimento que é de ser feito segundo as mesmas regras e com as mesmas consequências da união estável heteroafetiva".

§ 1 o A união estável não se constituirá se ocorrerem os impedimentos do art. 1.521; não se aplicando a incidência do inciso VI no caso de a pessoa casada se achar separada de fato ou judicialmente.


///

GABARITO CERTO, acredito que esteja errado.



A questão trata da solidariedade passiva, pois, no caso apresentado, o fornecedor tem o direito de exigir e receber de um ou de alguns dos devedores (João e Maria), parcial ou totalmente, a dívida comum. A respeito do tema, o art. 276 do Código Civil traz regra específica para a hipótese de morte de um dos devedores solidários: “Art. 276. Se um dos devedores solidários falecer deixando herdeiros, nenhum destes será obrigado a pagar senão a quota que corresponder ao seu quinhão hereditário, salvo se a obrigação for indivisível; mas todos reunidos serão considerados como um devedor solidário em relação aos demais devedores”.

O item está certo.

A questão trata da solidariedade passiva, pois, no caso apresentado, o fornecedor tem o direito de exigir e receber de um ou de alguns dos devedores (João e Maria), parcial ou totalmente, a dívida comum. A respeito do tema, o art. 276 do Código Civil traz regra específica para a hipótese de morte de um dos devedores solidários:



“Art. 276. Se um dos devedores solidários falecer deixando herdeiros, nenhum destes será obrigado a pagar senão a quota que corresponder ao seu quinhão hereditário, salvo se a obrigação for indivisível; mas todos reunidos serão considerados como um devedor solidário em relação aos demais devedores”.

///

Afirmativa correta, Letra E.

A justificativa se deve ao fato de que o seu teor reflete a interpretação atual do Superior Tribunal de Justiça (STJ) sobre a questão da paternidade socioafetiva.

A paternidade socioafetiva é um conceito jurídico que reconhece a relação de pai e filho baseada em laços de amor e afeto, não apenas em laços biológicos.

Isso significa que, mesmo que um homem não seja o pai biológico de uma criança, ele ainda pode ser reconhecido como o pai legal da criança se tiver uma relação de amor e cuidado com ela.

Portanto, em uma ação negatória de paternidade cumulada com pedido de anulação de registro (ou seja, um processo legal onde alguém está tentando provar que não é o pai de uma criança), o STJ exige que a pessoa demonstre a inexistência de uma relação socioafetiva com a criança.

Isso significa que a pessoa deve provar que não tem uma relação de amor e cuidado com a criança.

A razão para isso é proteger os melhores interesses da criança.

O STJ entende que a relação de amor e cuidado entre um pai e uma criança é mais importante do que a relação biológica.

Portanto, na linha de raciocínio do STJ, mesmo que alguém possa provar que não é o pai biológico de uma criança, ele ainda pode ser considerado o pai legal da criança se tiver uma relação socioafetiva com ela.

Com relação a culpa no divórcio, isso não existe mais.



O STF decidiu ano passado a respeito disso:



A tese de repercussão geral fixada para o Tema 1.053 é a seguinte:

“Após a promulgação da Emenda Constitucional 66/2010, a separação judicial não é mais requisito para o divórcio, nem subsiste como figura autônoma no ordenamento jurídico. Sem prejuízo, preserva-se o estado civil das pessoas que já estão separadas por decisão judicial ou escritura pública, por se tratar de um ato jurídico perfeito”.



Prevaleceu o entendimento do relator, ministro Luiz Fux, no sentido de que a alteração constitucional simplificou o rompimento do vínculo matrimonial e eliminou as condicionantes. Com isso, passou a ser inviável exigir separação judicial prévia para efetivar o divórcio, pois essa modalidade de dissolução do casamento deixou de depender de qualquer requisito temporal ou causal.

O STJ sedimentou o entendimento de que, em conformidade com os princípios do Código Civil e da CF de 1988, o sucesso da ação negatória de paternidade, depende da demonstração, a um só tempo:

a) da inexistência de origem biológica;

b) de que não tenha sido constituído o estado de filiação socioafetiva, edificado, na maioria das vezes, pela convivência familiar; e

c) demonstração inequívoca de vício de consentimento do pai registral no momento do registro.

Portanto, é de rigor a Improcedência da Ação Negatória de Paternidade, ainda que com DNA negativo, quando houver a Paternidade Socioafetiva.

(AGRAVO EM RECURSO ESPECIAL Nº 1.393.889 - MG (2018/0292717-8)

Nesse sentido, o STJ admitiu que um homem, que havia reconhecido a paternidade do filho de sua companheira por não suspeitar de nenhum adultério e que já havia formado vínculo socioafetivo nos 6 primeiros anos de vida da criança, pleiteasse a nulidade desse reconhecimento de paternidade logo após descobrir que a criança era filho de uma amante da companheira (STJ, REsp 1330404/RS, 3ª Turma, Rel. Ministro Marco Aurélio Bellizze, DJe 19/02/2015).

O mesmo raciocínio se aplica para o caso de adultério: o marido traído poderá impugnar a filiação que fora registrada com base na presunção de paternidade, desde que o faça logo que tome ciência da origem extraconjugal do filho. Não importa se houve ou não formação de vínculo socioafetivo, pois o art. 1.601 do CC garante o direito do marido traído.

FONTE APOSTILA DO GRAN

agora fiquei na dúvida

Toda família só é considerada uma se houver vínculo AFETIVO, não sendo necessário o vínculo sexual.


///


Código Civil, art. 270: Se um dos credores solidários falecer deixando herdeiros, cada um destes só terá direito a exigir e receber a quota do crédito que corresponder ao seu quinhão hereditário, salvo se a obrigação for indivisível.

A questão trata de tema relacionado com a solidariedade ativa.



Dispõe o art. 270 do Código Civil que: “Se um dos credores solidários falecer deixando herdeiros, cada um destes só terá direito a exigir e receber a quota do crédito que corresponder ao seu quinhão hereditário, salvo se a obrigação for indivisível”.



Analisando o dispositivo legal, Flávio Tartuce afirma que: “A premissa não deverá ser aplicada se a obrigação for naturalmente indivisível, como no exemplo da entrega de um animal para fins de reprodução ou de um veículo. Nesse caso, se um dos credores falecer, o cumprimento dessa obrigação indivisível ocorrerá se o objeto for entregue a qualquer um dos sucessores deste. É pertinente frisar que esse efeito não mantém relação com a solidariedade, mas sim com a indivisibilidade da obrigação”.

///

art. 1º da Resolução nº 35/2007:

Art. 1' Para a lavratura dos atos notariais de que trata a Lei no 11.441107, é livre a escolha do tabelião de notas, não se aplicando as regras de competência do Código de Processo Civil.

O inventário poderá ser aberto em qualquer lugar de escolha do tabelião de notas. A SUCESSÃO que será aberta no último domicílio do falecido.

///

CC:

Art. 1.813: Quando o renunciante prejudicar os seus credores, renunciando à herança, poderão eles, com autorização do juiz, aceitá-la em nome do renunciante.
Art. 1.813. Quando o herdeiro prejudicar os seus credores, renunciando à herança, poderão eles, com autorização do juiz, aceitá-la em nome do renunciante.

§ 1 o A habilitação dos credores se fará no prazo de trinta dias seguintes ao conhecimento do fato.

///

(A) INCORRETO:

Art. 1.860. Além dos incapazes, não podem testar os que, no ato de fazê-lo, não tiverem pleno discernimento.

Parágrafo único. Podem testar os maiores de dezesseis anos.



(B) CORRETO:

Art. 1.858. O testamento é ato personalíssimo, podendo ser mudado a qualquer tempo.



(C) INCORRETO:

Art. 1.786. A sucessão dá-se por lei ou por disposição de última vontade.



(D) INCORRETO:

Art. 1.845. São herdeiros necessários os descendentes, os ascendentes e o cônjuge.

Art. 1.860. Além dos incapazes, não podem testar os que, no ato de fazê-lo, não tiverem pleno discernimento.

Parágrafo único. Podem testar os maiores de dezesseis anos.

///

Ao cego só se permite o testamento público, que lhe será lido, em voz alta, duas vezes, uma pelo tabelião ou por seu substituto legal, e a outra por uma das testemunhas, designada pelo testador, fazendo-se de tudo circunstanciada menção no testamento (art. 1.867).

///

Art. 1.864. São requisitos essenciais do testamento público:

I - ser escrito por tabelião ou por seu substituto legal em seu livro de notas, de acordo com as declarações do testador, podendo este servir-se de minuta, notas ou apontamentos;



II - lavrado o instrumento, ser lido em voz alta pelo tabelião ao testador e a duas testemunhas, a um só tempo; ou pelo testador, se o quiser, na presença destas e do oficial;

///

Art. 1.844. Não sobrevivendo cônjuge, ou companheiro, nem parente algum sucessível, ou tendo eles renunciado a herança, esta se devolve ao Município ou ao Distrito Federal, se localizada nas respectivas circunscrições, ou à União, quando situada em território federal.

a) Art. 1.836. Na falta de descendentes, são chamados à sucessão os ascendentes, em concorrência com o cônjuge sobrevivente.



b) se não houver cônjuge sobrevivente, nas condições estabelecidas no art. 1.830, serão chamados a suceder os colaterais até o quarto grau.



d) Art. 1.838. Em falta de descendentes e ascendentes, será deferida a sucessão por inteiro ao cônjuge sobrevivente.



e)1843 § 2 o Se concorrem filhos de irmãos bilaterais com filhos de irmãos unilaterais, cada um destes herdará a metade do que herdar cada um daqueles.

///

Art. 1.785. A sucessão abre-se no lugar do último domicílio do falecido.

Consoante dispõe o art. 1.785 do CC, é no lugar do último domicilio do “de cujus", haja vista a presunção de lá se concentrar a maior parte das suas relações jurídicas.

Art. 1.785. A sucessão abre-se no lugar do último domicílio do falecido.

///

Art. 1.798. Legitimam-se a suceder as pessoas nascidas ou já concebidas no momento da abertura da sucessão.

Art. 1.799. Na sucessão testamentária podem ainda ser chamados a suceder:

I - os filhos, ainda não concebidos, de pessoas indicadas pelo testador, desde que vivas estas ao abrir-se a sucessão;

II - as pessoas jurídicas;

III - as pessoas jurídicas, cuja organização for determinada pelo testador sob a forma de fundação.

A questão não está incompleta. Quando se diz "só seria valido se ela já tivesse nascido" exclui qualquer outra possibilidade de indicação, o que não é verdade, dado que uma pessoa JÁ CONCEBIDA seria outra alternativa possível e válida para indicação de sucessão.

A questão não cita se a criança já foi concebida ou não. Achei incompleta a informação.

Não importa se o filho foi concebido ou não, na questão, o que importa é que o amigo esteja vivo no momento da sucessão.

///

Art. 1.696. O direito à prestação de alimentos é recíproco entre pais e filhos, e extensivo a todos os ascendentes, recaindo a obrigação nos mais próximos em grau, uns em falta de outros.

Art. 1.698. Se o parente, que deve alimentos em primeiro lugar, não estiver em condições de suportar totalmente o encargo, serão chamados a concorrer os de grau imediato; sendo várias as pessoas obrigadas a prestar alimentos, todas devem concorrer na proporção dos respectivos recursos, e, intentada ação contra uma delas, poderão as demais ser chamadas a integrar a lide.



Súmula 596 do STJ: "A obrigação alimentar dos avós tem natureza complementar e subsidiária, somente se configurando no caso de impossibilidade total ou parcial de seu cumprimento pelos pais."

E) Art. 1.707. Pode o credor não exercer, porém lhe é vedado renunciar o direito a alimentos, sendo o respectivo crédito insuscetível de cessão, compensação ou penhora.

Aprofundando:

JDC341 Para os fins do art. 1.696, a relação

socioafetiva pode ser elemento gerador de

obrigação alimentar.

///


Art. 1.573 - Podem caracterizar a impossibilidade da comunhão de vida a ocorrência de algum dos seguintes motivos:

I - adultério;

II - tentativa de morde;

III - sevícia ou injúria grave;

IV - abandono voluntário do lar conjugal, durante um ano contínuo;

V - condenação por crime infamante;

VI - conduta desonrosa.

Parágrafo único. O juiz poderá considerar outros fatos que tornem evidente a impossibilidade da vida em comum.

///

STJ (AgRg no Ag 1.212.745 / RJ): “(…) II – O bem integrante de herança jacente só é devolvido ao Estado com a sentença de declaração da vacância, podendo, até ali, ser possuído ad usucapionem. Incidência da Súmula 83/STJ. Agravo improvido”.

A - Como o Estado só adquire a propriedade declarada a vacância (aquisição da propriedade resolúvel pelo Estado), conforme explicado na assertiva D, nesse interregno estão sujeitos a usucapião tais bens, visto que não são do Estado e, portanto, não incorrem na restrição de usucapião de bens públicos.




B - CC/02, art. 1.844 - Não sobrevivendo cônjuge, ou companheiro, nem parente algum sucessível, ou tendo eles renunciado a herança, esta se devolve ao Município ou ao Distrito Federal, se localizada nas respectivas circunscrições, ou à União, quando situada em território federal.





C - CPC/15. art. 738 - Nos casos em que a lei considere jacente a herança, o juiz em cuja comarca tiver domicílio o falecido procederá imediatamente à arrecadação dos respectivos bens.





D - O Estado não é herdeiro, mas um sucessor irregular, não estando sujeito ao princípio de saisine. O momento da vacância não se confunde com o da abertura da sucessão ou da morte do de cujus.





E - CPC/15, art. 739 - A herança jacente ficará sob a guarda, a conservação e a administração de um curador até a respectiva entrega ao sucessor legalmente habilitado OU até a declaração de vacância.


"""

In [130]:
comentarios_pag2 = """
A:ERRADA. irmãos são parentes colaterais em segundo grau.

B:ERRADA. ART. 1.595: o parentesco por afinidade limita-se aos ascendentes, aos descendentes e aos irmãos do cônjuge ou companheiro.

C: ERRADA. Não há nenhum grau de parentesco entre cônjuges, há porém com seus parentes: pai, mãe, filho, até o limite de irmão.

D:CERTA. pela adoção, é constituído parentesco civil entre pais e filhos na linha reta em primeiro grau.

Só há parentesco de primeiro grau em linha reta, ou seja: ascendentes ou descendentes.

Parente por afinidade é aquele que será resultado do casamento, ou seja, os parentes do cônjuge, nesse sentindo, o artigo 1.595 do CC, em seu parágrafo primeiro, aduz que o parentesco por afinidade é exclusivo de ascendentes, descendentes e irmãos do companheiro.

Não existe grau de parentesco entre os cônjuges, mas sim, uma relação jurídica: o casamento.

Já, no que consta acerca do parentesco civil, pode-se salientar que qualquer relação que não seja consanguínea pode ser considerada parentesco civil, exemplo: adoção, maternidade e paternidade socioafetiva.

é aquele ditado: marido não é parente!!

///

ódigo Civil

Art. 136. O encargo não suspende a aquisição nem o exercício do direito, salvo quando expressamente imposto no negócio jurídico, pelo disponente, como condição suspensiva.

Sobre a letra D: Art. 137. Considera-se não escrito o encargo ilícito ou impossível, salvo se constituir o motivo determinante da liberalidade, caso em que se invalida o negócio jurídico.

O encargo necessariamente deve ser acessório a um negócio jurídico que expresse uma liberalidade. Por essa razão, não pode o encargo assumir a natureza de uma contraprestação sob pena de sua descaracterização. É o que ocorre, por exemplo, à doação de um imóvel com o encargo de que o donatário nele construa uma creche.

CORRETA: LETRA A

O testamento pode impor condições para recebimento, mas o direito de Laura se dá partir da abertura da sucessão.

Art. 1.897. A nomeação de herdeiro, ou legatário, pode fazer-se pura e simplesmente, sob condição, para certo fim

ou modo, ou por certo motivo.

O encargo é uma restrição imposta ao beneficiário, que deve ser lícita e possível.

Art. 136. O encargo não suspende a aquisição nem o exercício do direito, salvo quando expressamente imposto no negócio jurídico, pelo disponente, como condição suspensiva.



OPÇÃO B: ERRADA

Não sei explicar



OPÇÃO C: ERRADA

O encargo é previsto nos arts. 136 e 137 do CC



OPÇÃO D: ERRADA



Art. 123, CC/02: Invalidam os negócios jurídicos que lhes são subordinados:

I - as condições física ou juridicamente impossíveis, quando suspensivas;

II - as condições ilícitas, ou de fazer coisa ilícita;

III - as condições incompreensíveis ou contraditórias.



Art. 137. Considera-se não escrito o encargo ilícito ou impossível, salvo se constituir o motivo determinante da liberalidade, caso em que se invalida o negócio jurídico.



Opção E: ERRADA

Condição suspensiva é aquela que suspende os efeitos do negócio jurídico até o implemento da condição e, a partir dela o negócio produzirá efeitos (art. 125, CC).

No caso, a condição já existe que é cuidar do animal de Inês e não é mencionado em momento algum existência de termo suspensivo, que deve ser expresso. Dessa forma, sua amiga Laura adquire a herança desde a abertura da sucessão, não é necessário esperar o animal morrer.

acho o erro da alternativa B) está em "um evento futuro e certo". Parece ser um caso de condição (evento futuro e incerto) e não termo (evento futuro e certo). Pois não é certo que Laura vai cuidar do animal de Inês.

///

Art. 1.793. O direito à sucessão aberta, bem como o quinhão de que disponha o co-herdeiro, pode ser objeto de cessão por escritura pública.

§ 1 o Os direitos, conferidos ao herdeiro em conseqüência de substituição ou de direito de acrescer, presumem-se não abrangidos pela cessão feita anteriormente.

§ 2 o É ineficaz a cessão, pelo co-herdeiro, de seu direito hereditário sobre qualquer bem da herança considerado singularmente.

§ 3 o Ineficaz é a disposição, sem prévia autorização do juiz da sucessão, por qualquer herdeiro, de bem componente do acervo hereditário, pendente a indivisibilidade.

Art. 1.794. O co-herdeiro não poderá ceder a sua quota hereditária a pessoa estranha à sucessão, se outro co-herdeiro a quiser, tanto por tanto.

Art. 1.795. O co-herdeiro, a quem não se der conhecimento da cessão, poderá, depositado o preço, haver para si a quota cedida a estranho, se o requerer até cento e oitenta dias após a transmissão.

Parágrafo único. Sendo vários os co-herdeiros a exercer a preferência, entre eles se distribuirá o quinhão cedido, na proporção das respectivas quotas hereditárias.

I-Art. 1.794. O co-herdeiro não poderá ceder a sua quota hereditária a pessoa estranha à sucessão, se outro co-herdeiro a quiser, tanto por tanto.



II- 1.793

§ 2 o É ineficaz a cessão, pelo co-herdeiro, de seu direito hereditário sobre qualquer bem da herança considerado singularmente.



III-

Art. 1.793. O direito à sucessão aberta, bem como o quinhão de que disponha o co-herdeiro, pode ser objeto de cessão por escritura pública.

§ 1 o Os direitos, conferidos ao herdeiro em conseqüência de substituição ou de direito de acrescer, presumem-se não abrangidos pela cessão feita anteriormente.

///

Art. 1.830. Somente é reconhecido direito sucessório ao cônjuge sobrevivente se, ao tempo da morte do outro, não estavam separados judicialmente, nem separados de fato há mais de dois anos, salvo prova, neste caso, de que essa convivência se tornara impossível sem culpa do sobrevivente.

CÓDIGO CIVIL



Art. 25. O cônjuge do ausente, sempre que não esteja separado judicialmente, ou de fato por mais de dois anos antes da declaração da ausência, será o seu legítimo curador.



§ 1 o Em falta do cônjuge, a curadoria dos bens do ausente incumbe aos pais ou aos descendentes, nesta ordem, não havendo impedimento que os iniba de exercer o cargo.



§ 2 o Entre os descendentes, os mais próximos precedem os mais remotos.



§ 3 o Na falta das pessoas mencionadas, compete ao juiz a escolha do curador.

///

a) Herdeiros necessários podem ser excluídos da sucessão em razão de indignidade (art. 1.814 do CC/02) ou deserdação (art. 1.962 do CC/02);



b) Quando o motivo da indignidade dizer relação a crime, a exclusão é condicionada a sentença penal condenatória transitada em julgado. Após a sentença penal, a exclusão é automática (art. 1.815-A do CC/02);



c) Sim, ele será excluído, entretanto, a partir de 2023 com o art. 1.815-A do CC/02, a exclusão é automática à sentença penal condenatória transitada em julgado;



d) Segundo o art. 1.815, p. 1°, o prazo para demandar a exclusão do herdeiro ou legatário por indignidade é de 4 anos, contados da abertura da sucessão;



e) Letra da lei: Art. 1.816. São pessoais os efeitos da exclusão; os descendentes do herdeiro excluído sucedem, como se ele morto fosse antes da abertura da sucessão.

///

Código Civil



"Art. 1.802. São nulas as disposições testamentárias em favor de pessoas não legitimadas a suceder, ainda quando simuladas sob a forma de contrato oneroso, ou feitas mediante interposta pessoa.

Parágrafo único. Presumem-se pessoas interpostas os ascendentes, os descendentes, os irmãos e o cônjuge ou companheiro do não legitimado a suceder."

///

Gabarito: C





"1.No regime de separação convencional de bens, o cônjuge sobrevivente concorre com os descendentes do falecido, sendo apenas afastada a concorrência quanto ao regime de separação legal de bens previsto no art. 1.641, do Código Civil (REsp 1830753/RJ)"



Quanto aos filhos da filha falecida, esses receberão a parte correspondente da herança, como representantes, como diz o Art. 1.851 CC

RESPOSTA : C

ART. 1829 E ART. .1840 DO CÓDIGO CIVIL.

OS FILHOS DE JUSTINA HERDÃO POR REPRESENTAÇÃO.

enunciado 527 da V Jornada de direito civil. “Na concorrência entre o cônjuge e os herdeiros do de cujus, não será reservada a quarta parte da herança para o sobrevivente no caso de filiação híbrida”

///

CC. Art. 1.858. O testamento é ato personalíssimo, podendo ser mudado a qualquer tempo.

Um testamento pode ser modificado ou revogado pelo testador, total ou parcialmente, a qualquer momento, por meio de outro testamento.

Irrevogável é o erro da questão

O testamento é Revogável

O testamento é ato revogável visto que o testador poderá, por meio de outro testamento, modificar ou revogar o anterior (art. 1.858).

///

artigo 1863, CC, que: “é proibido o testamento conjuntivo, seja simultâneo, recíproco ou correspectivo”. É chamado de testamento conjuntivo aquele feito por duas ou mais pessoas, no mesmo ato.

Art. 1.863. É proibido o testamento conjuntivo, seja simultâneo, recíproco ou correspectivo.

Por se tratar o testamento de ato individual e personalíssimo, proíbe-se a realização de testamento conjuntivo ou de mão comum, que é aquele em que duas ou mais pessoas, por um único instrumento, no mesmo ato de última vontade, dispõem de seus bens.
O testamento conjuntivo é simultâneo quando os testadores fazem disposições comuns em benefício de terceiros;
O testamento conjuntivo é recíproco quando cada um dos testadores beneficia o outro, de modo que o sobrevivente recolha a herança do outro;
O testamento conjuntivo é correspectivo quando, além da reciprocidade, cada testador beneficia o outro na mesma proporção em que aquele o tiver beneficiado.
A intenção da lei é garantir a plena liberdade do testador, manter viva a possibilidade de revogação do testamento, a qualquer momento, e evitar pactos sucessórios.

///

CC - Art. 1.867. Ao cego só se permite o testamento público, que lhe será lido, em voz alta, duas vezes, uma pelo tabelião ou por seu substituto legal, e a outra por uma das testemunhas, designada pelo testador, fazendo-se de tudo circunstanciada menção no testamento.

naão são duas pelo tabelião

///

Marcos: não há nenhum impedimento p/ a sucessão
Jéssica (filha de Mirtes): não há nenhum impedimento p/ a sucessão, com base no art. 1816, do CC (São pessoais os efeitos da exclusão; os descendentes do herdeiro excluído sucedem, como se ele morto fosse antes da abertura da sucessão.)
Augusto (filho de Mário): como seu pai renunciou, ele não pode receber por representação, de acordo com art. 1811, do CC (Ninguém pode suceder, representando herdeiro renunciante.)
Ana: não pode receber, pois "o direito de representação, na sucessão colateral, está limitado aos filhos dos irmãos", conforme art. 1853, do CC (Na linha transversal, somente se dá o direito de representação em favor dos filhos de irmãos do falecido, quando com irmãos deste concorrerem). No caso, Ana é neta da irmã da falecida.

Mariana morreu em 2020, e Maitê e seu único filho Igor faleceram em acidente de carro no ano de 2018





Art. 1.798. Legitimam-se a suceder as pessoas nascidas ou já concebidas no momento da abertura da sucessão.

///

Institui o Código Civil.

Art. 1.814. São excluídos da sucessão os herdeiros ou legatários:

I - que houverem sido autores, co-autores ou partícipes de homicídio doloso, ou tentativa deste, contra a pessoa de cuja sucessão se tratar, seu cônjuge, companheiro, ascendente ou descendente;

II - que houverem acusado caluniosamente em juízo o autor da herança ou incorrerem em crime contra a sua honra, ou de seu cônjuge ou companheiro;

III - que, por violência ou meios fraudulentos, inibirem ou obstarem o autor da herança de dispor livremente de seus bens por ato de última vontade.

///

Na indignidade a exclusão depende de sentença transitada em julgado, proferida pelo juízo das sucessões; na deserdação, exige-se a prévia manifestação do autor da herança em testamento, que será homologado em juízo (homologação do testamento, submetida ao procedimento de jurisdição voluntária).

O ato infracional análogo ao homicídio, doloso e consumado, praticado contra os pais, pode ser sim incluído na regra do art. 1.814, I, do CC.

O rol do art. 1.814 do CC/2002 é taxativo, não admitindo analogia ou interpretação extensiva. Contudo, o fato de o rol ser taxativo não significa que ele só possa ser interpretado literalmente. Não se pode confundir taxatividade com interpretação literal.

Mesmo que um rol seja taxativo, ele pode ser objeto de outras formas de interpretação, como a interpretação lógica, a histórica, a sistemática, a teleológica, a sociológica etc.

O herdeiro que seja autor, coautor ou partícipe de ato infracional análogo ao homicídio doloso praticado contra os ascendentes fica excluído da sucessão.

STJ. 3ª Turma. REsp 1.943.848-PR, Rel. Min. Nancy Andrighi, julgado em 15/02/2022 (Info 725).

A alternativa correta é a letra C.

A alternativa A está incorreta. O STF fixou entendimento no sentido de ser possível a hipótese de deserção em caso de ato infracional análogo ao homicídio doloso. Nestes termos: “Registra-se que a exclusão do herdeiro que atenta contra a vida dos pais, cláusula geral com raiz ética, moral e jurídica existente desde o direito romano, está presente na maioria dos ordenamentos jurídicos contemporâneos e, no Brasil, possui, como núcleo essencial, a exigência de que a conduta ilícita do herdeiro seja dolosa, ainda que meramente tentada, sendo irrelevante investigar se a motivação foi ou não o recolhimento da herança. A finalidade da regra que exclui da sucessão o herdeiro que atenta contra a vida dos pais é, a um só tempo, prevenir a ocorrência do ato ilícito, tutelando bem jurídico mais valioso do ordenamento jurídico, e reprimir o ato ilícito porventura praticado, estabelecendo sanção civil consubstanciado na perda do quinhão por quem praticá-lo. Assim, se o enunciado normativo do art. 1.814, I, do CC/2002, na perspectiva teleológica-finalística, é de que não terá direito à herança quem atentar, propositalmente, contra a vida de seus pais, ainda que a conduta não se consuma, independentemente do motivo, a diferença técnico-jurídica entre o homicídio doloso e o ato análogo ao homicídio doloso, conquanto relevante para o âmbito penal diante das substanciais diferenças nas consequências e nas repercussões jurídicas do ato ilícito, não se reveste da mesma relevância no âmbito civil, sob pena de ofensa aos valores e às finalidades que nortearam a criação da norma e de completo esvaziamento de seu conteúdo” (STJ. 3ª Turma. REsp 1943848-PR, Rel. Min. Nancy Andrighi, julgado em 15/02/2022 - Info 725).

A alternativa B está incorreta. Não se trata de hipótese de deserção, mas sim de indignidade, nos termos do art. 1.814, inciso I, do CC: “Art. 1.814. São excluídos da sucessão os herdeiros ou legatários: I - que houverem sido autores, co-autores ou partícipes de homicídio doloso, ou tentativa deste, contra a pessoa de cuja sucessão se tratar, seu cônjuge, companheiro, ascendente ou descendente.”

Alternativa C está correta. Alternativa que está de acordo com o julgado e com a legislação supracitada.

Alternativa D está incorreto. Assertiva está incorreta por afirmar não ser possível a interpretação extensiva do art. 1.814, inciso I, do CC para incluir a hipótese de ato infracional a homicídio doloso.

Alternativa E está incorreta. O erro está em afirmar que Mariazinha não poderá receber a herança.

Indignidade: Ato judicial

Deserdação: Testamento

Art. 1.815-A. Em qualquer dos casos de indignidade previstos no art. 1.814, o trânsito em julgado da sentença penal condenatória acarretará a imediata exclusão do herdeiro ou legatário indigno, independentemente da sentença prevista no caput do art. 1.815 deste Código. (Incluído pela Lei nº 14.661, de 2023)

FUNDAMENTAÇÃO conforme artigo 1816 do CC.





Art. 1.816. São pessoais os efeitos da exclusão; os descendentes do herdeiro excluído sucedem, como se ele morto fosse antes da abertura da sucessão.

Parágrafo único. O excluído da sucessão não terá direito ao usufruto ou à administração dos bens que a seus sucessores couberem na herança, nem à sucessão eventual desses bens.


///

ECA, art. 26. Os filhos havidos fora do casamento poderão ser reconhecidos pelos pais, conjunta ou separadamente, no próprio termo de nascimento, por testamento, mediante escritura ou outro documento público, qualquer que seja a origem da filiação.



CC, art. 1.609. O reconhecimento dos filhos havidos fora do casamento é irrevogável e será feito:

III - por testamento, ainda que incidentalmente manifestado;



CC, art. 1.610. O reconhecimento não pode ser revogado, nem mesmo quando feito em testamento.



CC, art. 1.849. O herdeiro necessário, a quem o testador deixar a sua parte disponível, ou algum legado, não perderá o direito à legítima.

Pra colaborar, fiquem ligados!

A regra que estabelece tratamento diferenciado na sucessão é no caso de COLATERAIS! Assim, IRMÃOS bilaterais recebem o dobro dos unilaterais. Em relação ao enunciado, a questão trata de sucessão de ascendente, e não de irmão.

É FÁCIL, BASTA INTERPRETAÇÃO DO ART. 1841 DO CC, IRMÃO DO FALECIDO E NA QUESTÃO SÃO FILHOS DO FALECIDO.

///

Os nascituros possuem vocação hereditária, ou seja, podem ser chamados a suceder tanto na sucessão legítima como na testamentária, ficando a eficácia da vocação dependente do seu nascimento.



Permite-se, no inciso I, do art. 1.799, do CC, que seja feita disposição testamentária a favor de prole eventual, isto é, filhos de determinadas pessoas designadas pelo testador. A única exigência que a lei traz no caso de prole eventual é que as pessoas indicadas pelo testador estejam vivas quando da abertura da sucessão.

A) CORRETO

Art. 1.798. Legitimam-se a suceder as pessoas nascidas ou já concebidas no momento da abertura da sucessão.

Art. 1.799. Na sucessão testamentária podem ainda ser chamados a suceder:

I - os filhos, ainda não concebidos, de pessoas indicadas pelo testador, desde que vivas estas ao abrir-se a sucessão;

II - as pessoas jurídicas;

III - as pessoas jurídicas, cuja organização for determinada pelo testador sob a forma de fundação.

B) ERRADO

Art. 1.843. Na falta de irmãos, herdarão os filhos destes e, não os havendo, os tios.

C) ERRADO

A primeira parte da assertiva está correta. De fato, não há representação no caso de sucessão testamentária, pois se o legatário falecer caducará o legado.

Art. 1.939. Caducará o legado:

V - se o legatário falecer antes do testador.

O equívoco da segunda parte da assertiva está no art. 1853 do CC.

Art. 1.853. Na linha transversal, somente se dá o direito de representação em favor dos filhos de irmãos do falecido, quando com irmãos deste concorrerem.

D) ERRADO

Parece pertinente uma leitura conjunta do art. 1857 do Código Civil com o art. 6º do Estatuto da Pessoa com Deficiência:

Art. 1.857. Toda pessoa capaz pode dispor, por testamento, da totalidade dos seus bens, ou de parte deles, para depois de sua morte.

Art. 6º A deficiência não afeta a plena capacidade civil da pessoa, inclusive para:

Qualquer equívoco de minha parte, podem retificar nos comentários. Bons estudos.

NO MEU VER 2 RESPOSTA CERTAS

Art. 1.857. Do CC

Toda pessoa capaz pode dispor, por testamento, da totalidade dos seus bens, ou de parte deles, para depois de sua morte.

O direito de representação não se aplica a sucessão testamentária, o art 1851, CC fala de parentes do falecido, mas um bem pode ser legado para alguém que não é parente, nesse caso não há que se falar em direito de representação, a alternativa C está certa

///

LETRA A



Súmula 596 do STJ: "A obrigação alimentar dos avós tem natureza complementar e subsidiária, somente se configurando no caso de impossibilidade total ou parcial de seu cumprimento pelos pais."



SÚMULA N. 358 O cancelamento de pensão alimentícia de filho que atingiu a maioridade está sujeito à decisão judicial, mediante contraditório, ainda que nos próprios autos.



Súmula 621 do Superior Tribunal de Justiça: Os efeitos da sentença que reduz, majora ou exonera o alimentante do pagamento retroagem à data da citação, vedadas a compensação e a repetibilidade.



SÚMULA 309 O débito alimentar que autoriza a prisão civil do alimentante é o que compreende as três prestações anteriores ao ajuizamento da execução e as que se vencerem no curso do processo.



Súmula 596 do STJ: "A obrigação alimentar dos avós tem natureza complementar e subsidiária, somente se configurando no caso de impossibilidade total ou parcial de seu cumprimento pelos pais."

Complementando o comentário da colega:

Julgada procedente a investigação de paternidade, os alimentos são devidos a partir da citação.

(Súmula 277, SEGUNDA SEÇÃO, julgado em 14/05/2003, DJ 16/06/2003 p. 416)



Vamo com tudo, como quem não quer nada! <3

///

GABARITO LETRA D

(I) CORRETO.

Enunciado 632 do CJF: Nos casos de reconhecimento de multiparentalidade paterna ou materna, o filho terá direito à participação na herança de todos os ascendentes reconhecidos.

(II) INCORRETO. O STJ consolidou orientação no sentido de que para ser possível a anulação do registro de nascimento, é imprescindível a presença de dois requisitos, a saber:

1- prova robusta no sentido de que o pai foi de fato induzido a erro, ou ainda, que tenha sido coagido a tanto e

2- inexistência de relação socioafetiva entre pai e filho. Assim, a divergência entre a paternidade biológica e a declarada no registro de nascimento não é apta, por si só, para anular o registro. REsp 1829093/PR. Precedentes.

(III) CORRETO. De acordo com o Provimento nº 63/2017 do Conselho Nacional de Justiça, o registro de nascimento de crianças geradas por meio da técnica de reprodução assistida de gestação por substituição poderá ser solicitado no Cartório de Registro Civil, não sendo necessária qualquer intervenção judicial.

(IV) CORRETO. O STF possui firme entendimento no sentido de que a paternidade socioafetiva, declarada ou não em registro público, não impede o reconhecimento do vínculo de filiação concomitante baseado na origem biológica, com todas as suas consequências jurídicas. (RE 898060).

///

Certo!



Art. 1.857 do CC - Toda pessoa capaz pode dispor, por testamento, da totalidade dos seus bens, ou de parte deles, para depois de sua morte.

§ 1º A legítima dos herdeiros necessários não poderá ser incluída no testamento.

§ 2º São válidas as disposições testamentárias de caráter não patrimonial, ainda que o testador somente a elas se tenha limitado.

///

A ausência de prévia propositura de ação de investigação de paternidade, imprescritível, e de seu julgamento definitivo não constitui óbice para o ajuizamento de ação de petição de herança e para o início da contagem do prazo prescricional.

A definição da paternidade e da afronta ao direito hereditário, na verdade, apenas interfere na procedência da ação de petição de herança.

O termo inicial do prazo prescricional da pretensão de petição de herança conta-se da abertura da sucessão, ou, em se tratando de herdeiro absolutamente incapaz, da data em que completa 16 (dezesseis) anos, momento em que, em ambas as hipóteses, nasce para o herdeiro, ainda que não legalmente reconhecido, o direito de reivindicar os direitos sucessórios (actio nata).

STJ. 2ª Seção. EAREsp 1260418/MG, Rel. Min. Antonio Carlos Ferreira, julgado em 26/10/2022 (Info 757).



Prazo prescricional

A pretensão de petição de herança prescreve no prazo de 10 anos, nos termos do art. 205 do CC, já que não existe um prazo específico fixado no Código:

Art. 205. A prescrição ocorre em dez anos, quando a lei não lhe haja fixado prazo menor.

Os herdeiros NECESSÁRIOS podem reclamar a herança vacante em até 05 anos da abertura da sucessão (morte do de cujus). No caso em questão, quando Renato protocolou a ação de petição de herança em 15/12/22, já havia prescrito, uma vez que ultrapassou os cinco anos previsto na lei, pois a abertura da sucessão (ato contínuo ao falecimento) ocorreu em 05/02/2010.



Ver artigo 1.822, do CC: "A declaração de vacância da herança não prejudicará os herdeiros que legalmente se habilitarem; mas, decorridos cinco anos da abertura da sucessão, os bens arrecadados passarão ao domínio do Município ou do Distrito Federal, se localizados nas respectivas circunscrições, incorporando-se ao domínio da União quando situados em território federal."

O STF editou a súmula 149: "É imprescritível a ação de investigação de paternidade, mas não o é a da petição de herança". Neste sentido, sua prescrição é de 10 anos, conforme artigo 205 do CC.



O STF também determinou que o prazo prescricional tem início com a abertura da sucessão. Todavia, a petição de herança depende do reconhecimento de paternidade post mortem, tendo entendimento fixado no sentido que o dies a quo do prazo prescricional será a data em que o direito puder ser exercido, ou seja, a partir do trânsito em julgado da ação de investigação de paternidade, quando restará confirmada a sua condição de herdeiro, e não o da abertura da sucessão. Entretanto, em 2019, ocorreu novo julgamento na 4ª Turma do STJ onde voltou a ser aplicado o prazo prescricional a partir do início da abertura da sucessão. Assim, a questão aguarda pacificação.

Galera o erro está em "apenas posteriormente, optou por tomar medida jurídica para que houvesse o reconhecimento da paternidade e a restituição de herança."

se ele tivesse ingressado com ação assim que soube, o cenário mudaria. Mas ele sabendo da situação optou em não agir. E o direito não socorre a aqueles que dormem.

O resto o pessoal já explicou.

A ausência de prévia propositura da ação de investigação de paternidade, que é imprescritível, e de seu julgamento definitivo não constitui obstáculo para o início da contagem do prazo prescricional para o ajuizamento da ação de petição de herança.

Além disso, o STJ entende que se a sucessão ocorreu antes do Código Civil de 2002, o prazo prescricional será de 20 anos. Se posterior, será de 10 anos.


///

certo.





Art. 1.798. Legitimam-se a suceder as pessoas nascidas ou já concebidas no momento da abertura da sucessão

Complementando a resposta do colega, o Código Civil também prevê a legitimidade para suceder de pessoas ainda não concebidas.



Art. 1.799. Na sucessão testamentária podem ainda ser chamados a suceder: I. os filhos, ainda não concebidos, de pessoas indicadas pelo testador, desde que vivas estas ao abrir-se a sucessão;


"""

In [91]:
comentarios_pag1 = comentarios_pag1.split("///")
len(comentarios_pag1)

20

In [131]:
comentarios_pag2 = comentarios_pag2.split("///")
len(comentarios_pag2)

20

In [134]:
comentarios = comentarios_pag1 + comentarios_pag2
for idx, comentario in enumerate(comentarios):
  comentario = comentario.strip()
  comentario = comentario.replace("\n\n", "\n")
  comentario = comentario.replace("\n\n", "\n")
  comentarios[idx] = comentario
  print('-----------')
  print(comentario)



-----------
CC
Art. 1.826. O possuidor da herança está obrigado à restituição dos bens do acervo, fixando-se-lhe a responsabilidade segundo a sua posse, observado o disposto nos arts. 1.214 a 1.222.
Parágrafo único. A partir da citação, a responsabilidade do possuidor se há de aferir pelas regras concernentes à posse de má-fé e à mora.
Art. 1.214. O possuidor de boa-fé tem direito, enquanto ela durar, aos frutos percebidos.
Parágrafo único. Os frutos pendentes ao tempo em que cessar a boa-fé devem ser restituídos, depois de deduzidas as despesas da produção e custeio; devem ser também restituídos os frutos colhidos com antecipação.
Art. 1.219. O possuidor de boa-fé tem direito à indenização das benfeitorias necessárias e úteis, bem como, quanto às voluptuárias, se não lhe forem pagas, a levantá-las, quando o puder sem detrimento da coisa, e poderá exercer o direito de retenção pelo valor das benfeitorias necessárias e úteis.
Art. 1.973. Sobrevindo descendente sucessível ao testador, qu

## Join Data

In [137]:
question_df = pd.DataFrame(output_list)
# question_df.loc[0, 'pergunta_editada'] = question_df.loc[0, 'pergunta_editada'].split(':')[1].strip()

# temp = question_df.iloc[0].copy()
# question_df.iloc[0] = question_df.iloc[1].copy()
# question_df.iloc[1] = temp.copy()
# del temp

# temp = question_df.iloc[4].copy()
# question_df.iloc[4] = question_df.iloc[5].copy()
# question_df.iloc[5] = temp.copy()
# del temp

# temp = question_df.iloc[11].copy()
# question_df.iloc[11] = question_df.iloc[12].copy()
# question_df.iloc[12] = temp.copy()
# del temp

# temp = question_df.iloc[11].copy()
# question_df.iloc[11] = question_df.iloc[13].copy()
# question_df.iloc[13] = temp.copy()
# del temp

# temp = question_df.iloc[16].copy()
# question_df.iloc[16] = question_df.iloc[14].copy()
# question_df.iloc[14] = temp.copy()
# del temp

# temp = question_df.iloc[16].copy()
# question_df.iloc[16] = question_df.iloc[15].copy()
# question_df.iloc[15] = temp.copy()
# del temp

data_df = {
    'pergunta': question_df['pergunta_editada'],
    'resposta': respostas,
    'header': headers.split('\n\n\n'),
    'comentarios': comentarios
}

data_df = pd.DataFrame(data_df)
data_df

,pergunta,resposta,header,comentarios
0,"Diego, em razão do falecimento do seu pai Eucl...",C,\n1\nQ3116412\nDireito Civil Direito das Suce...,CC\nArt. 1.826. O possuidor da herança está ob...
1,A sucessão testamentária é um componente impor...,B,2\nQ3121863\nDireito Civil Sucessão Testament...,I -\nArt. 1.867. Ao cego só se permite o testa...
2,"Matheus, filho de um milionário, aos dezesseis...",E,3\nQ3251846\nDireito Civil Sucessão Testament...,A resposta da questão está no Art. 1.880. O te...
3,"Mariana, solteira, não convivente em união est...",C,4\nQ3158359\nDireito Civil Sucessão Testament...,"Art. 1.860. Além dos incapazes, não podem test..."
4,Lofredo faleceu deixando quatro filhas vivas: ...,E,5\nQ3116405\nDireito Civil Direito das Sucess...,Essa questão não tem gabarito certo. Explico.\...
5,Rogério é casado com Cláudio e ambos são pais ...,C,6\nQ3192677\nDireito Civil Direito de Família...,Gabarito C.\n\nA preexistência de casamento ou...
6,"João e Maria, amigos de longa data, decidem em...",E,7\nQ3231292\nDireito Civil Direito das Sucess...,"GABARITO CERTO, acredito que esteja errado.\nA..."
7,"Em março de 2003, Homero e Agatha se casaram p...",E,8\nQ3116416\nDireito Civil Direito de Família...,"Afirmativa correta, Letra E.\nA justificativa ..."
8,"João e Maria, amigos de longa data, decidem em...",E,9\nQ3231293\nDireito Civil Direito das Sucess...,"Código Civil, art. 270: Se um dos credores sol..."
9,"Fernando, viúvo, faleceu em Brasília, seu últi...",E,10\nQ3071800\nDireito Civil Direito das Suces...,art. 1º da Resolução nº 35/2007:\nArt. 1' Para...


In [145]:
new_row = {'numero_na_captura': None, 'id': None, 'area': None, 'ano':None, 'banca': None, 'prova': None}
new_rows = []
for sample in data_df['header']:
    if sample[0] == '\n':
        sample = sample[1:]
    splitted = sample.split('\n')
    new_row['numero_na_captura'] = splitted[0]
    new_row['id'] = splitted[1]
    new_row['area'] = splitted[2]
    new_row['ano'] = splitted[3].split(':')[1]
    new_row['banca'] = splitted[4].split(':')[1]
    new_row['prova'] = splitted[5].split(':')[1]
    new_rows.append(new_row.copy())

append_df = pd.DataFrame(new_rows)
append_df

,numero_na_captura,id,area,ano,banca,prova
0,1,Q3116412,Direito Civil Direito das Sucessões (Art. 1.7...,2024,Fundação Getúlio Vargas - FGV,FGV - Câmara de São Paulo - Procurador Legisl...
1,2,Q3121863,Direito Civil Sucessão Testamentária (Art. 1....,2024,Instituto Consulplan,Instituto Consulplan - DPE PR - Analista - Ár...
2,3,Q3251846,Direito Civil Sucessão Testamentária (Art. 1....,2024,Fundação para o Vestibular da Universidade Es...,VUNESP - MPE RO - Promotor de Justiça Substit...
3,4,Q3158359,Direito Civil Sucessão Testamentária (Art. 1....,2024,Fundação Getúlio Vargas - FGV,FGV - MPE GO - Promotor de Justiça Substituto...
4,5,Q3116405,Direito Civil Direito das Sucessões (Art. 1.7...,2024,Fundação Getúlio Vargas - FGV,FGV - Câmara de São Paulo - Procurador Legisl...
5,6,Q3192677,Direito Civil Direito de Família (Art. 1.511 ...,2024,Fundação para o Vestibular da Universidade Es...,VUNESP - MPE RJ - Promotor de Justiça Substit...
6,7,Q3231292,Direito Civil Direito das Sucessões (Art. 1.7...,2024,Instituto Consulplan,Instituto Consulplan - MPE SC - Promotor de J...
7,8,Q3116416,Direito Civil Direito de Família (Art. 1.511 ...,2024,Fundação Getúlio Vargas - FGV,FGV - Câmara de São Paulo - Procurador Legisl...
8,9,Q3231293,Direito Civil Direito das Sucessões (Art. 1.7...,2024,Instituto Consulplan,Instituto Consulplan - MPE SC - Promotor de J...
9,10,Q3071800,Direito Civil Direito das Sucessões (Art. 1.7...,2023,Fundação Getúlio Vargas - FGV,FGV - Câmara dos Deputados - Analista Legisla...


In [146]:
data_df = pd.concat([data_df, append_df], axis=1)
data_df = data_df.drop('header', axis=1)
data_df

,pergunta,resposta,comentarios,numero_na_captura,id,area,ano,banca,prova
0,"Diego, em razão do falecimento do seu pai Eucl...",C,CC\nArt. 1.826. O possuidor da herança está ob...,1,Q3116412,Direito Civil Direito das Sucessões (Art. 1.7...,2024,Fundação Getúlio Vargas - FGV,FGV - Câmara de São Paulo - Procurador Legisl...
1,A sucessão testamentária é um componente impor...,B,I -\nArt. 1.867. Ao cego só se permite o testa...,2,Q3121863,Direito Civil Sucessão Testamentária (Art. 1....,2024,Instituto Consulplan,Instituto Consulplan - DPE PR - Analista - Ár...
2,"Matheus, filho de um milionário, aos dezesseis...",E,A resposta da questão está no Art. 1.880. O te...,3,Q3251846,Direito Civil Sucessão Testamentária (Art. 1....,2024,Fundação para o Vestibular da Universidade Es...,VUNESP - MPE RO - Promotor de Justiça Substit...
3,"Mariana, solteira, não convivente em união est...",C,"Art. 1.860. Além dos incapazes, não podem test...",4,Q3158359,Direito Civil Sucessão Testamentária (Art. 1....,2024,Fundação Getúlio Vargas - FGV,FGV - MPE GO - Promotor de Justiça Substituto...
4,Lofredo faleceu deixando quatro filhas vivas: ...,E,Essa questão não tem gabarito certo. Explico.\...,5,Q3116405,Direito Civil Direito das Sucessões (Art. 1.7...,2024,Fundação Getúlio Vargas - FGV,FGV - Câmara de São Paulo - Procurador Legisl...
5,Rogério é casado com Cláudio e ambos são pais ...,C,Gabarito C.\n\nA preexistência de casamento ou...,6,Q3192677,Direito Civil Direito de Família (Art. 1.511 ...,2024,Fundação para o Vestibular da Universidade Es...,VUNESP - MPE RJ - Promotor de Justiça Substit...
6,"João e Maria, amigos de longa data, decidem em...",E,"GABARITO CERTO, acredito que esteja errado.\nA...",7,Q3231292,Direito Civil Direito das Sucessões (Art. 1.7...,2024,Instituto Consulplan,Instituto Consulplan - MPE SC - Promotor de J...
7,"Em março de 2003, Homero e Agatha se casaram p...",E,"Afirmativa correta, Letra E.\nA justificativa ...",8,Q3116416,Direito Civil Direito de Família (Art. 1.511 ...,2024,Fundação Getúlio Vargas - FGV,FGV - Câmara de São Paulo - Procurador Legisl...
8,"João e Maria, amigos de longa data, decidem em...",E,"Código Civil, art. 270: Se um dos credores sol...",9,Q3231293,Direito Civil Direito das Sucessões (Art. 1.7...,2024,Instituto Consulplan,Instituto Consulplan - MPE SC - Promotor de J...
9,"Fernando, viúvo, faleceu em Brasília, seu últi...",E,art. 1º da Resolução nº 35/2007:\nArt. 1' Para...,10,Q3071800,Direito Civil Direito das Sucessões (Art. 1.7...,2023,Fundação Getúlio Vargas - FGV,FGV - Câmara dos Deputados - Analista Legisla...


In [147]:
data_df.to_csv('dataset_concursos.csv', index=False)

## Agente com JSON RAG Search

In [159]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("data").load_data()
documents

[Document(id_='3b307d6e-c91f-4430-bf7d-58e07e95115d', embedding=None, metadata={'file_path': '/content/data/codigo_civil.json', 'file_name': 'codigo_civil.json', 'file_type': 'application/json', 'file_size': 845418, 'creation_date': '2024-06-24', 'last_modified_date': '2024-06-24'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='{\n    "introducao": "\\nPresidência da República\\nCasa Civil\\nSubchefia para Assuntos Jurídicos\\n\\nLEI Nº 10.406, DE 10 DE JANEIRO DE 2002\\n\\nTexto compilado\\n\\nÍNDICE\\n\\nVigência\\n\\nLei de Introdução às normas do Direito Brasileiro\\n\\n(Vide Lei nº 14.195, de 2021)\\n\\nVide Lei nº 14.451, de 2022   Vigência\\n\\nInstitui o Código Civil.\\n\\nO PRESIDENTE DA REPÚBLICA Faço saber que o Congresso Nacional decreta e 

In [164]:
from llama_index.core import Settings, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq


Settings.embed_model = HuggingFaceEmbedding(model_name='rufimelo/Legal-BERTimbau-sts-large-ma-v3')
Settings.llm = Groq(model="llama3-70b-8192", temperature=0, api_key=GROQ_API_KEY)
index = VectorStoreIndex.from_documents(
    documents,
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.21k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/900 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/678k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Invalid model-index. Not loading eval results into CardData.


1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Invalid model-index. Not loading eval results into CardData.


In [169]:
query_engine = index.as_query_engine()
response = query_engine.query("O que é doação?")
print(response)

Based on the provided context, there is no direct mention of "doação" (donation). However, we can infer that the context is related to the Brazilian Civil Code, and "doação" is a concept related to the transfer of ownership or property.

In the context of the Brazilian Civil Code, "doação" can be understood as the act of transferring ownership or property from one person to another without receiving anything in return. This concept is not explicitly mentioned in the provided context, but it is related to the ideas of property, ownership, and transfer of rights, which are discussed in the code.

In the context of Article 62, which discusses the creation of a foundation, it is mentioned that the founder must make a special endowment of free assets, specifying the purpose of the foundation. This can be seen as a form of donation, where the founder transfers ownership of the assets to the foundation.

In summary, while "doação" is not directly mentioned in the context, it can be inferred a

In [168]:
index.storage_context.persist('.')